#### Image Generator

This program is primarily concerned with the generation of OOD images. In particular, it seeks to find the bounds for the images in each cluster and generates images accordingly. It is able to generate images associated with insertion. It also generates the intermediate images for the deletion and rotation operations. LaMa is then used to further  manipulate the image.

In [ ]:
# !pip3 install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
torch.cuda.is_available()
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
# !pip3 install 'git+https://github.com/facebookresearch/detectron2.git'
# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, random, cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageChops
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
def scale_rotate_image(img, img_scale=0.75, img_rotate=30):
    bg_w, bg_h = img.size
    return img.resize((int(bg_w*img_scale), int(bg_h*img_scale))).rotate(img_rotate, expand=True)

In [ ]:
def gen_random_image(offset_low = 1, offset_high = 1, fg_scale = 0.5, fg_rot = -45, 
                     bg_scale = 1, bg_rotate = 0):
    car = scale_rotate_image(Image.open("./Tesla.png", 'r'), 
                             img_scale = fg_scale, img_rotate=fg_rot)
    stop = scale_rotate_image(Image.open("./stop.png", 'r'), 
                             img_scale = 0.05, img_rotate=0)
    background = scale_rotate_image(Image.open("./road_background.jpg", 'r'), 
                                    img_scale = bg_scale, img_rotate = bg_rotate)
    img_w, img_h = car.size

    bg_w, bg_h = background.size
    o1 = random.randint(offset_low, offset_high)
    o2 = random.randint(offset_low, offset_high)
    offset = ((bg_w - img_w) // o1 - 100, (bg_h - img_h) // o2 - 60)
    
    offset2 = (30, 60)
    
    background.paste(car, offset, mask=car)
    # background.paste(stop, offset2, mask=stop)
    return np.array(background.resize((640, 480)).convert('RGB'))

In [ ]:
def countObj(segments_info):

    import json
    with open('Categories.txt') as f:
        catList = json.load(f)
        
    #classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']
    classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'train', 'building', 'tree']

    classesShrink = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']
    
    classScale = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck']
    classIndexes = [0, 1, 2, 3, 5, 7]
    classScale = [1.05, 0.5, 1.0, 0.5, 2.1, 1.2]
    
    scale = [1.05, 1.0, 1.1, 1.2, 0.2, 0.2, 0.25]
    
    classNum = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for item in segments_info:

        itemInd = item['category_id']

        isThingCheck = item['isthing']

        if not isThingCheck:

            itemInd = itemInd + 80

        curType = catList[itemInd]['name']

        if curType in classes:

            index = classes.index(curType)
            classNum[index] = classNum[index] + 1
            
    return classNum

In [ ]:
import math

def findScale(image):
    
    imageWidth, imageHeight = image.size
    
    image = np.array(image.convert('RGB'))
    
    classesShrink = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']
    #scaleObjTypeList = [1.0, 1.0, 0.9, 1.1, 0.25, 0.25, 0.28]
    
    classScale = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck']
    classIndexes = [0, 1, 2, 3, 5, 7]
    classScale = [1.05, 0.5, 1.0, 0.5, 2.1, 1.2]
    
    scale = [1.05, 1.0, 1.1, 1.2, 0.2, 0.2, 0.25]
    
    cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)
    
    predClasses = outputs["instances"].pred_classes
    predBoxes = outputs["instances"].pred_boxes
    
    index = -1
    count = 0
    
    for predClass in predClasses:
        
        index = index + 1
        
        #person
        if (predClass==0):
            
            count = count + 1
            
            predBox = predBoxes[index]
            predBox = predBox.tensor.tolist()
            predBox = predBox[0]
            topVal = predBox[1]
            botVal = predBox[3]
            
            height = botVal - topVal
            height = math.floor(height/1.0)
            pos = math.floor(botVal)
            
        #bicycle
        elif (predClass==1):
            
            count = count + 1
            
            predBox = predBoxes[index]
            predBox = predBox.tensor.tolist()
            predBox = predBox[0]
            topVal = predBox[1]
            botVal = predBox[3]
            
            height = botVal - topVal
            height = math.floor(height/0.5)
            pos = math.floor(botVal)
            
        #car
        elif (predClass==2):
            
            count = count + 1
            
            predBox = predBoxes[index]
            predBox = predBox.tensor.tolist()
            predBox = predBox[0]
            topVal = predBox[1]
            botVal = predBox[3]
            
            height = botVal - topVal
            height = math.floor(height)
            pos = math.floor(botVal)
            
        #motorcycle
        elif (predClass==3):
            
            count = count + 1
            
            predBox = predBoxes[index]
            predBox = predBox.tensor.tolist()
            predBox = predBox[0]
            topVal = predBox[1]
            botVal = predBox[3]
            
            height = botVal - topVal
            height = math.floor(height/0.5)
            pos = math.floor(botVal)
            
        #bus
        elif (predClass==5):
            
            count = count + 1
            
            predBox = predBoxes[index]
            predBox = predBox.tensor.tolist()
            predBox = predBox[0]
            topVal = predBox[1]
            botVal = predBox[3]
            
            height = botVal - topVal
            height = math.floor(height/2.1)
            pos = math.floor(botVal)
            
        #truck
        elif (predClass==7):
            
            count = count + 1
            
            predBox = predBoxes[index]
            predBox = predBox.tensor.tolist()
            predBox = predBox[0]
            topVal = predBox[1]
            botVal = predBox[3]
            
            height = botVal - topVal
            height = math.floor(height/1.1)
            pos = math.floor(botVal)
            
        if count==1:
            
            oneHeight = height
            onePos = pos
        elif count==2:
            
            twoHeight = height
            twoPos = pos
            break
            
    if count==1:
        
        if onePos>0.75*imageHeight:
            
            twoPos = math.floor(0.5*imageHeight)
            twoHeight = math.floor(oneHeight*0.33)
        else:
            
            twoPos = math.floor(0.9*imageHeight)
            if math.floor(0.35*imageHeight)>oneHeight:
                twoHeight = math.floor(0.35*imageHeight)
            else:
                twoHeight = math.floor(oneHeight*3)
            
    if count==0:
        
        onePos = math.floor(0.5*imageHeight)
        oneHeight = math.floor(0.1*imageHeight)
        
        twoPos = math.floor(0.9*imageHeight)
        twoHeight = math.floor(0.35*imageHeight)
    
    if onePos>twoPos:
        
        intPos = onePos
        onePos = twoPos
        twoPos = intPos
        intHeight = oneHeight
        oneHeight = twoHeight
        twoHeight = intHeight
        
    if twoPos==onePos:
        
        twoPos = onePos+1
        
    if twoHeight<oneHeight:
        
        if onePos>0.75*imageHeight:
            
            twoPos = math.floor(0.5*imageHeight)
            twoHeight = math.floor(oneHeight*0.33)
            
            intPos = onePos
            onePos = twoPos
            twoPos = intPos
            intHeight = oneHeight
            oneHeight = twoHeight
            twoHeight = intHeight
        else:
            
            twoPos = math.floor(0.9*imageHeight)
            if math.floor(0.35*imageHeight)>oneHeight:
                twoHeight = math.floor(0.35*imageHeight)
            else:
                twoHeight = math.floor(oneHeight*3)
        
        #image.show()
        #print("Error")
        
    scaleHeight = twoHeight-oneHeight
    scalePos = twoPos-onePos
    
    scaleCur = scaleHeight/scalePos
    
    scale50 = math.floor(scaleCur*(math.floor(0.5*imageHeight)-onePos) + oneHeight)
    
    return (scale50, scaleCur)

In [ ]:
'''
scaleInfoList = []

for i in range(1):

    curNum = str(i).zfill(6)

    strName = "bremen_" + curNum + "_000019_leftImg8bit.png"

    openName = "./BremenCase/" + strName

    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)

    background1 = background1.resize((2048, 1024))

    #cfg = get_cfg()
    #cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    #predictor = DefaultPredictor(cfg)

    curImg = np.array(background1.convert('RGB'))
    #panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

    #print(panoptic_seg)
    #print(segments_info)

    #scale50, scaleCur = findScale(curImg)
    tupleScale = findScale(background1)
    
    scaleInfoList.append(tupleScale)
    
print(scaleInfoList)
'''

scaleInfoList = []

for i in range(300):

    strName = "COCO" + str(i) + ".jpg"
    
    openName = "./Clean/MSCOCO/Raw/" + strName

    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)

    #cfg = get_cfg()
    #cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    #predictor = DefaultPredictor(cfg)

    #curImg = np.array(background1.convert('RGB'))
    #panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

    #print(panoptic_seg)
    #print(segments_info)

    #scale50, scaleCur = findScale(curImg)
    tupleScale = findScale(background1)
    
    scaleInfoList.append(tupleScale)
    
print(scaleInfoList)


In [ ]:
'''
import time

initTime = time.perf_counter()

finDist = []

for i in range(315):
    
    curNum = str(i).zfill(6)
    
    strName = "bremen_" + curNum + "_000019_leftImg8bit.png"
    
    openName = "./BremenCase/" + strName
    
    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
    
    background1 = background1.resize((2048, 1024))
        
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    predictor = DefaultPredictor(cfg)

    curImg = np.array(background1.convert('RGB'))
    panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

    countObjs = countObj(segments_info)
    
    finDist.append(countObjs)
        
        
finTime = time.perf_counter()

elapTime = finTime - initTime

print(elapTime)
'''

import time

initTime = time.perf_counter()

finDist = []

for i in range(300):
    
    strName = "COCO" + str(i) + ".jpg"
    
    openName = "./Clean/MSCOCO/Raw/" + strName
    
    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
    
    #background1 = background1.resize((2048, 1024))
        
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    predictor = DefaultPredictor(cfg)

    curImg = np.array(background1.convert('RGB'))
    panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

    countObjs = countObj(segments_info)
    
    finDist.append(countObjs)
        
        
finTime = time.perf_counter()

elapTime = finTime - initTime

print(elapTime)

In [ ]:
print(finDist)

In [ ]:
print(finDist)

maxList = []

for index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
    personCount = []
    
    for elem in finDist:

        curCount = elem[index]
        personCount.append(curCount)

    maxPerson = max(personCount)
    
    maxList.append(maxPerson)

    #print(maxPerson)
    
print((maxList))

In [ ]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
from google.cloud import vision
#%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw,ImageColor,ExifTags
from io import BytesIO
import sys

secretText = 'SECRETS.txt'

fSecret = open(secretText, "r")

while True:
    
    line = fSecret.readline()
    
    if not line:
        break
    
    words = line[:-1]
        
    if words.startswith('GCP_JSON'):
        
        start = words.find('[') + 1
        
        end = words.find(']')
        
        intWords = words[start:end]
        
        gcpJSON = intWords

In [ ]:
def getRoadID(segments_info):

    import json
    with open('Categories.txt') as f:
        catList = json.load(f)

    itemIDs = []

    for item in segments_info:

        itemInd = item['category_id']

        isThingCheck = item['isthing']

        if not isThingCheck:

            itemInd = itemInd + 80

        curType = catList[itemInd]['name']

        if curType == 'road' or curType == 'floor' or curType =='pavement' or curType == 'grass' or curType =='dirt':

            itemID = item['id']
            itemIDs.append(itemID)
            
    return itemIDs

In [ ]:
def getRoadIDCar(segments_info):

    import json
    with open('Categories.txt') as f:
        catList = json.load(f)

    itemIDs = []

    for item in segments_info:

        itemInd = item['category_id']

        isThingCheck = item['isthing']

        if not isThingCheck:

            itemInd = itemInd + 80

        curType = catList[itemInd]['name']

        if curType == 'road' or curType =='dirt':

            itemID = item['id']
            itemIDs.append(itemID)
            
    return itemIDs

In [ ]:
def getBoundTransparent(img):
    
    imgWidth, imgHeight = img.size
    
    topVal = 0
    botVal = 0
    leftVal = 0
    rightVal = 0
    
    found = False
    
    for indH in range(imgHeight):
        for indW in range(imgWidth):
            
            if found == False:
            
                red, green, blue, alpha = img.getpixel((indW, indH))

                if alpha!=0:
                    
                    #print(indH)
                    #print(indW)

                    found = True
                    topVal = indH
                
    found = False
                
    for indH in range(imgHeight):
        for indW in range(imgWidth):
            
            if found == False:
                
                curHeight = imgHeight - indH - 1
                curWidth = imgWidth - indW - 1
            
                red, green, blue, alpha = img.getpixel((curWidth, curHeight))

                if alpha!=0:

                    found = True
                    botVal = curHeight
                    
    found = False
                    
    for indW in range(imgWidth):
        for indH in range(imgHeight):
            
            if found == False:
            
                red, green, blue, alpha = img.getpixel((curWidth, curHeight))

                if alpha!=0:

                    found = True
                    leftVal = indW
                    
    found = False
                    
    for indW in range(imgWidth):
        for indH in range(imgHeight):
            
            if found == False:
                
                curHeight = imgHeight - indH - 1
                curWidth = imgWidth - indW - 1
            
                red, green, blue, alpha = img.getpixel((curWidth, curHeight))

                if alpha!=0:

                    found = True
                    rightVal = curWidth
                    
    boundBox = (topVal, botVal, leftVal, rightVal)
    
    return boundBox

In [ ]:
def getSegmentID(segments_info, classType):

    import json
    with open('Categories.txt') as f:
        catList = json.load(f)

    itemIDs = []

    for item in segments_info:

        itemInd = item['category_id']

        isThingCheck = item['isthing']

        if not isThingCheck:

            itemInd = itemInd + 80

        curType = catList[itemInd]['name']

        if curType == classType:

            itemID = item['id']
            itemIDs.append(itemID)
            
    return itemIDs

In [ ]:
def getSegmentIDSingle(segments_info, classType):

    import json
    with open('Categories.txt') as f:
        catList = json.load(f)

    itemIDs = []

    for item in segments_info:

        itemInd = item['category_id']

        isThingCheck = item['isthing']

        if not isThingCheck:

            itemInd = itemInd + 80

        curType = catList[itemInd]['name']

        if curType == classType:

            itemID = item['id']
            itemIDs.append(itemID)
            
    chosenId = []
    
    if(itemIDs):
        
        randomChoice = random.choice(itemIDs)
        chosenId.append(randomChoice)
            
    return chosenId

In [ ]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
#%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw,ImageColor,ExifTags
from io import BytesIO
import sys

secretText = 'SECRETS.txt'

fSecret = open(secretText, "r")

while True:
    
    line = fSecret.readline()
    
    if not line:
        break
    
    words = line[:-1]
        
    if words.startswith('MS_Key'):
        
        start = words.find('[') + 1
        
        end = words.find(']')
        
        intWords = words[start:end]
        
        msKey = intWords

In [ ]:
subscription_key = "Key-Here"
assert subscription_key

# You must use the same region in your REST call as you used to get your
# subscription keys. For example, if you got your subscription keys from
# westus, replace "westcentralus" in the URI below with "westus".
#
# Free trial subscription keys are generated in the "westus" region.
# If you use a free trial subscription key, you shouldn't need to change
# this region.
# vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/"
vision_base_url = "https://eastus2.api.cognitive.microsoft.com/vision/v2.0/"

analyze_url = vision_base_url + "analyze"

# Set image_path to the local path of an image that you want to analyze.
# image_path = "/Users/XXX/work/project/testing-object-detector/data/kite.png"
image_path = "./bremen.png"

# Read the image into a byte array
image_data = open(image_path, "rb").read()
headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
              'Content-Type': 'application/octet-stream'}
params     = {'visualFeatures': 'Objects'}
response = requests.post(
    analyze_url, headers=headers, params=params, data=image_data)
response.raise_for_status()

# The 'analysis' object contains various fields that describe the image. The most
# relevant caption for the image is obtained from the 'description' property.
analysis = response.json()
# print(analysis)
# image_caption = analysis["description"]["captions"][0]["text"].capitalize()

# Display the image and overlay it with the caption.

image = Image.open(open(image_path, "rb"))
imgWidth, imgHeight = image.size
draw = ImageDraw.Draw(image)

# calculate and display bounding boxes for each detected face
for label in analysis['objects']:
    # print (label)
    print (label)
    box = label['rectangle']
    left = box['x']
    top = box['y']
    width = box['w']
    height = box['h']

    points = (
        (left,top),
        (left + width, top),
        (left + width, top + height),
        (left , top + height),
        (left, top)
    )
    #print (label['object'], points)
    #if label['object'] == "Person":
    draw.line(points, fill='#00d400', width=2)

    # Alternatively can draw rectangle. However you can't set line width.
    #draw.rectangle([left,top, left + width, top + height], outline='#00d400')

image.show()

for label in analysis['objects']:
    
    objectMS = label['object']
    print(objectMS)

In [ ]:
insertObj(background, inObj, scale50, scaleCur, posH, posW):
    
    

In [ ]:

groupData = [0,0,0,0,0,2,1,0,0,0,1,0,1,0,0,0,1,1,2,2,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,2,2,0,0,0,0,0,1,0,1,2,1,0,0
,1,0,0,0,1,2,0,0,1,0,1,0,0,0,0,0,0,2,0,2,0,2,2,2,1,2,1,1,1,1,1,2,0,0,2,2,2
,2,2,2,0,2,2,0,2,0,0,0,2,2,2,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,1,0,0,0,0,0,0
,0,0,2,2,0,0,0,2,0,0,1,0,0,0,0,2,2,0,2,2,0,0,0,2,2,2,0,2,1,2,2,2,2,0,0,0,0
,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,2,2,0,2,2,0,0,1,2,2,0,2,2,2,2,0,1,2,2,0,0
,2,2,2,2,0,2,2,2,0,0,0,0,2,0,2,2,0,0,2,2,2,2,0,2,2,2,2,2,0,0,2,0,1,1,1,1,1
,0,2,0,0,0,0,2,0,0,0,2,2,0,0,0,2,2,2,0,2,2,0,0,0,0,0,0,1,1,2,2,0,0,1,2,2,0
,0,0,0,0,2,2,0,1,1,1,2,0,0,0,1,0,2,2,1]


'''
groupData = [2,2,1,2,1,2,1,0,1,1,1,0,2,1,1,1,3,1,0,1,2,3,1,1,3,3,2,3,2,2,3,
             1,1,2,2,1,3,1,3,1,1,2,3,2,2,1,2,2,3,1,1,3,1,0,1,1,3,1,2,3,1,3,0,
             2,1,0,2,1,1,3,1,2,1,1,1,2,1,3,3,2,1,3,2,1,1,3,1,3,3,0,0,1,0,1,3,
             3,1,3,1,3,2,2,1,2,1,2,1,1,1,1,3,2,1,1,1,0,1,1,2,2,3,1,0,2,2,3,1,3,
             1,2,0,1,1,2,1,1,1,1,3,3,1,3,1,1,2,0,1,2,3,1,1,2,2,1,1,1,2,1,1,1,2,
             1,1,2,3,2,2,2,1,1,3,2,1,3,1,1,3,1,1,3,2,3,1,3,1,1,1,1,1,3,1,1,3,
             1,2,3,2,1,3,1,3,3,2,2,1,2,1,1,0,3,2,0,2,3,3,3,3,1,1,1,2,1,1,1,1,
             1,1,0,1,1,2,2,1,3,1,3,2,1,3,2,1,1,1,0,2,3,1,1,1,1,1,2,2,1,1,1,1,3,
             3,2,1,3,1,2,1,1,1,1,1,3,1,1,1,1,1,1,1,3,3,2,1,1,1,3,3,1,3,1,1,3,3,
             0,1,1,1,2,1,1,1,1]
'''

print(len(groupData))
print(type(groupData))

In [ ]:
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []

for i in range(315):
    
    curGroup = groupData[i]
    
    if curGroup==0:
        
        cluster1.append(i)
    elif curGroup==1:
        
        cluster2.append(i)
    elif curGroup==2:
        
        cluster3.append(i)
    '''
    elif curGroup==3:
        
        cluster4.append(i)
    '''
        
print(cluster1)
print(cluster2)
print(cluster3)
#print(cluster4)

In [ ]:
'''
import time
import math
import json

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']

classesShrink = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']

classType = 'dog'

classInd = 9

orgCount = 0
rotCount = 0
rotErrCount = 0
rotMSErrCount = 0

msList = []
gcpList = []

initTime = time.perf_counter()

#315
for i in range(315):
    
    curNum = str(i).zfill(6)
    
    strName = "bremen_" + curNum + "_000019_leftImg8bit"
    
    strNameAdd = strName + ".png"
    
    openName = "./BremenCase/" + strNameAdd
    
    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
    
    background1 = background1.resize((2048, 1024))
    
    backgroundNew = background1
    
    image_path = openName
    
    client = vision.ImageAnnotatorClient().from_service_account_json(gcpJSON)
          
    image_data = open(image_path, "rb").read()
    headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
                  'Content-Type': 'application/octet-stream'}
    params     = {'visualFeatures': 'Objects'}
    response = requests.post(
        analyze_url, headers=headers, params=params, data=image_data)
    response.raise_for_status()
            
    analysis = response.json()
            
    orgDictMS = {}

    for label in analysis['objects']:

        key = label['object']
        key = key.lower()

        if key in orgDictMS:

            orgDictMS[key] = orgDictMS[key] + 1
        else:

            orgDictMS[key] = 1
            
    msList.append(orgDictMS)

    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    ig = vision.Image(content=content)

    objects = client.object_localization(
        image=ig).localized_object_annotations

    orgDict = {}

    for object_ in objects:

        key = object_.name
        key = key.lower()

        if key in orgDict:

            orgDict[key] = orgDict[key] + 1
        else:

            orgDict[key] = 1
            
    gcpList.append(orgDict)
    
    with open('Streets_GCP.txt', 'a') as file:
        
        file.write(openName + "\n")
        file.write(json.dumps(orgDict))
        file.write("\n")
        
    with open('Streets_MS.txt', 'a') as file:
        
        file.write(openName + "\n")
        file.write(json.dumps(orgDictMS))
        file.write("\n")
            
finTime = time.perf_counter()
elapTime = finTime-initTime

print(elapTime)

print(msList)
print(gcpList)
'''

import time
import math
import json

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']

classesShrink = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']

classType = 'dog'

classInd = 9

orgCount = 0
rotCount = 0
rotErrCount = 0
rotMSErrCount = 0

msList = []
gcpList = []

initTime = time.perf_counter()

#315
for i in range(300):
    
    strName = "COCO" + str(i) + ".jpg"
    
    openName = "./Clean/MSCOCO/Raw/" + strName
    
    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
    
    backgroundNew = background1
    
    image_path = openName
    
    client = vision.ImageAnnotatorClient().from_service_account_json(gcpJSON)
          
    image_data = open(image_path, "rb").read()
    headers    = {'Ocp-Apim-Subscription-Key': subscription_key,
                  'Content-Type': 'application/octet-stream'}
    params     = {'visualFeatures': 'Objects'}
    response = requests.post(
        analyze_url, headers=headers, params=params, data=image_data)
    response.raise_for_status()
            
    analysis = response.json()
            
    orgDictMS = {}

    for label in analysis['objects']:

        key = label['object']
        key = key.lower()

        if key in orgDictMS:

            orgDictMS[key] = orgDictMS[key] + 1
        else:

            orgDictMS[key] = 1
            
    msList.append(orgDictMS)

    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    ig = vision.Image(content=content)

    objects = client.object_localization(
        image=ig).localized_object_annotations

    orgDict = {}

    for object_ in objects:

        key = object_.name
        key = key.lower()

        if key in orgDict:

            orgDict[key] = orgDict[key] + 1
        else:

            orgDict[key] = 1
            
    gcpList.append(orgDict)
    
    with open('MSCOCO_GCP.txt', 'a') as file:
        
        file.write(openName + "\n")
        file.write(json.dumps(orgDict))
        file.write("\n")
        
    with open('MSCOCO_MS.txt', 'a') as file:
        
        file.write(openName + "\n")
        file.write(json.dumps(orgDictMS))
        file.write("\n")
            
finTime = time.perf_counter()
elapTime = finTime-initTime

print(elapTime)

print(msList)
print(gcpList)

In [ ]:
import boto3
import io

import time
import math
import json

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']

classesShrink = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']

awsList = []

initTime = time.perf_counter()

#315
for i in range(315):
    
    curNum = str(i).zfill(6)
    
    strName = "bremen_" + curNum + "_000019_leftImg8bit"
    
    strNameAdd = strName + ".png"
    
    openName = "./BremenCase/" + strNameAdd
    
    '''
    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
    
    background1 = background1.resize((2048, 1024))
    
    backgroundNew = background1
    '''
    
    image_path = openName
    
    image = Image.open(open(image_path,'rb'))
    row,col = image.size
    stream = io.BytesIO()
    image.save(stream, format=image.format)
    image_binary = stream.getvalue()

    client = boto3.client('rekognition', region_name='Region-Here', aws_access_key_id='Key-Here', 
                          aws_secret_access_key='Key-Here')
    response = client.detect_labels(Image={'Bytes': image_binary})
    
    orgDictAWS = {}
    
    for label in response['Labels']:
        
        for box in label['Instances']:                
        
            curObj = label['Name']
            key = curObj.lower()

            if key in orgDictAWS:

                orgDictAWS[key] = orgDictAWS[key] + 1
            else:

                orgDictAWS[key] = 1
            
    awsList.append(orgDictAWS)
    
    with open('Streets_AWS.txt', 'a') as file:
        
        file.write(openName + "\n")
        file.write(json.dumps(orgDictAWS))
        file.write("\n")
        
finTime = time.perf_counter()
elapTime = finTime-initTime

print(elapTime)

print(awsList)
    

In [ ]:
import boto3
import io

import time
import math
import json

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']

classesShrink = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']

awsList = []

initTime = time.perf_counter()

#315
for i in range(300):
    
    strName = "COCO" + str(i) + ".jpg"
    
    openName = "./Clean/MSCOCO/Raw/" + strName
    
    image_path = openName
    
    image = Image.open(open(image_path,'rb'))
    row,col = image.size
    stream = io.BytesIO()
    image.save(stream, format=image.format)
    image_binary = stream.getvalue()

    client = boto3.client('rekognition', region_name='Region-Here', aws_access_key_id='Key-Here', 
                          aws_secret_access_key='Key-Here')
    response = client.detect_labels(Image={'Bytes': image_binary})
    
    orgDictAWS = {}
    
    for label in response['Labels']:
        
        for box in label['Instances']:                
        
            curObj = label['Name']
            key = curObj.lower()

            if key in orgDictAWS:

                orgDictAWS[key] = orgDictAWS[key] + 1
            else:

                orgDictAWS[key] = 1
            
    awsList.append(orgDictAWS)
    
    with open('MSCOCO_AWS.txt', 'a') as file:
        
        file.write(openName + "\n")
        file.write(json.dumps(orgDictAWS))
        file.write("\n")
        
finTime = time.perf_counter()
elapTime = finTime-initTime

print(elapTime)

print(awsList)

In [ ]:
testList = awsList

In [ ]:
with open('Streets_GCP.txt', 'r') as file:
    
    testList = []
    
    ind = 1
    
    for line in file:
        
        if ind==-1:
            
            curDict = json.loads(line)
            testList.append(curDict)
            
        ind = ind*-1
        
#booleanListChecker = testList==gcpList
print(testList)
#print(gcpList)
#print(booleanListChecker)

In [ ]:
maxGcpDict = {}
maxMsDict = {}

for item in msList:
    
    for objType in item:
    
        if objType in maxMsDict:
            
            if item[objType] > maxMsDict[objType]:
                
                maxMsDict[objType] = item[objType]
                
        else:
            
            maxMsDict[objType] = item[objType]
                
for item in gcpList:
    
    for objType in item:
    
        if objType in maxGcpDict:
            
            if item[objType] > maxGcpDict[objType]:
                
                maxGcpDict[objType] = maxGcpDict[objType]
                
        else:
            
            maxGcpDict[objType] = item[objType]
                
print(maxGcpDict)
print(maxMsDict)

In [ ]:
maxDictGCP1 = {}
maxDictGCP2 = {}
maxDictGCP3 = {}
#maxDictGCP4 = {}

for i in cluster1:
    
    curDict = testList[i]
    
    for objType in curDict:
    
        if objType in maxDictGCP1:
            
            if curDict[objType] > maxDictGCP1[objType]:
                
                maxDictGCP1[objType] = curDict[objType]
                
        else:
            
            maxDictGCP1[objType] = curDict[objType]
            
for i in cluster2:
    
    curDict = testList[i]
    
    for objType in curDict:
    
        if objType in maxDictGCP2:
            
            if curDict[objType] > maxDictGCP2[objType]:
                
                maxDictGCP2[objType] = curDict[objType]
                
        else:
            
            maxDictGCP2[objType] = curDict[objType]
            
for i in cluster3:
    
    curDict = testList[i]
    
    for objType in curDict:
    
        if objType in maxDictGCP3:
            
            if curDict[objType] > maxDictGCP3[objType]:
                
                maxDictGCP3[objType] = curDict[objType]
                
        else:
            
            maxDictGCP3[objType] = curDict[objType]
            
'''
for i in cluster4:
    
    curDict = testList[i]
    
    for objType in curDict:
    
        if objType in maxDictGCP4:
            
            if curDict[objType] > maxDictGCP4[objType]:
                
                maxDictGCP4[objType] = curDict[objType]
                
        else:
            
            maxDictGCP4[objType] = curDict[objType]
'''
            
print(maxDictGCP1)
print(maxDictGCP2)
print(maxDictGCP3)
#print(maxDictGCP4)

In [ ]:
scaleInfoList = [(86, 0.4230769230769231), (96, 1.1288888888888888), (97, 0.6375), (128, 2.45), (150, 1.0560747663551402), (87, 1.141025641025641), (119, 1.0), (133, 1.9130434782608696), (151, 1.87012987012987), (63, 1.2763419483101393), (123, 1.146694214876033), (65, 1.726027397260274), (78, 2.5839416058394162), (91, 0.9239766081871345), (182, 2.75), (200, 0.38421052631578945), (106, 1.6111111111111112), (78, 1.45), (64, 1.1904761904761905), (85, 1.0), (122, 1.0), (89, 4.066666666666666), (120, 1.0289855072463767), (173, 1.1), (70, 0.7030456852791879), (114, 0.5949656750572082), (102, 0.7272727272727273), (69, 2.3405797101449277), (-17, 5.4), (145, 0.6521739130434783), (302, 0.13580246913580246), (104, 2.753623188405797), (112, 0.8301158301158301), (129, 2.75), (42, 2.0789473684210527), (108, 1.625), (354, 10.2), (168, 1.2444444444444445), (109, 1.170731707317073), (236, 0.29683698296836986), (131, 1.0747126436781609), (-26, 6.0285714285714285), (-46, 4.6875), (102, 1.0357142857142858), (138, 2.65), (91, 2.1842105263157894), (154, 0.5740740740740741), (85, 1.488888888888889), (-216, 2.7531645569620253), (72, 2.107142857142857), (58, 1.3157894736842106), (96, 1.0), (160, 2.0105263157894737), (99, 2.25), (-8, 2.980392156862745), (-4, 2.2777777777777777), (18, 1.8181818181818181), (115, 0.5357142857142857), (138, 1.5735294117647058), (60, 1.3300653594771241), (125, 0.5681818181818182), (133, 0.9032258064516129), (107, 0.6131386861313869), (39, 1.8444444444444446), (117, 1.2105263157894737), (-238, 3.5977011494252875), (72, 0.6976190476190476), (234, 0.6707317073170732), (84, 1.0307692307692307), (98, 1.3945945945945946), (78, 1.45), (120, 1.532258064516129), (70, 1.1710526315789473), (109, 2.4900662251655628), (135, 0.5436893203883495), (144, 0.5213032581453634), (136, 1.3397260273972602), (47, 0.9454545454545454), (52, 2.892857142857143), (168, 2.375), (27, 2.7857142857142856), (66, 0.9230769230769231), (141, 1.0909090909090908), (125, 1.1090909090909091), (126, 2.15), (195, 0.8787878787878788), (21, 1.0), (-94, 2.347826086956522), (109, 1.4190476190476191), (109, 1.2530364372469636), (216, 0.30597014925373134), (112, 0.0), (143, 0.5240506329113924), (92, 1.1578947368421053), (107, 3.5625), (41, 2.2), (117, 1.0625), (140, 0.5925925925925926), (161, 0.8235294117647058), (101, 1.6993865030674846), (101, 1.7755102040816326), (107, 1.59375), (187, 0.8732394366197183), (332, 0.06273062730627306), (110, 0.9732142857142857), (189, 0.5063291139240507), (130, 1.2692307692307692), (110, 1.291044776119403), (143, 0.38095238095238093), (336, 0.05263157894736842), (114, 0.15384615384615385), (70, 1.1951219512195121), (74, 1.17351598173516), (98, 1.0), (97, 1.169811320754717), (103, 0.7647058823529411), (108, 0.7021276595744681), (77, 1.3098591549295775), (91, 1.462686567164179), (92, 1.8571428571428572), (244, 5.380952380952381), (97, 2.5), (107, 1.1794871794871795), (42, 1.3103448275862069), (-102, 2.772727272727273), (67, 1.2607594936708861), (113, 1.124223602484472), (86, 1.050561797752809), (107, 1.25), (91, 0.8636363636363636), (93, 1.141732283464567), (78, 1.1694915254237288), (147, 1.068062827225131), (98, 1.3548387096774193), (73, 0.9285714285714286), (67, 1.2160493827160495), (96, 0.8115942028985508), (87, 1.0769230769230769), (130, 0.3225806451612903), (83, 0.96), (154, 0.4964705882352941), (100, 2.4), (77, 0.6857142857142857), (101, 1.3333333333333333), (115, 1.4242424242424243), (59, 1.8), (380, 14.0), (45, 1.7301587301587302), (104, 2.1739130434782608), (47, 0.625), (76, 1.32), (82, 1.2087912087912087), (249, 0.29411764705882354), (99, 1.0694444444444444), (99, 1.1), (102, 1.0), (106, 1.1486988847583643), (96, 1.1940298507462686), (70, 1.02), (85, 0.6153846153846154), (75, 1.4496644295302012), (24, 1.2), (136, 1.1911764705882353), (110, 0.8461538461538461), (106, 1.0212765957446808), (118, 1.4074074074074074), (100, 0.9261744966442953), (114, 0.923728813559322), (156, 0.5284552845528455), (84, 1.5245901639344261), (97, 1.1025641025641026), (96, 1.7142857142857142), (97, 3.2888888888888888), (109, 1.446808510638298), (109, 1.1791044776119404), (73, 1.1219512195121952), (170, 4.7), (108, 1.5106382978723405), (150, 0.5073529411764706), (163, 0.5072463768115942), (99, 1.1272727272727272), (96, 1.0222222222222221), (138, 1.0), (87, 1.0978260869565217), (99, 2.4375), (136, 0.7654320987654321), (84, 1.0333333333333334), (104, 4.571428571428571), (109, 1.4705882352941178), (106, 1.006993006993007), (242, 0.2823529411764706), (114, 1.0851063829787233), (109, 1.0769230769230769), (64, 1.6666666666666667), (74, 1.2436974789915967), (92, 1.1756756756756757), (-6, 1.8775510204081634), (71, 1.1), (126, 1.7058823529411764), (225, 0.3244047619047619), (120, 0.5811764705882353), (66, 1.6666666666666667), (151, 1.0703125), (31, 1.6176470588235294), (98, 0.9305555555555556), (73, 2.2782608695652176), (78, 1.88), (56, 1.2191235059760956), (-289, 13.0), (-89, 3.8), (-911, 8.142857142857142), (108, 0.8214285714285714), (120, 0.7884615384615384), (-433, 12.0), (91, 1.062330623306233), (51, 1.2058111380145278), (101, 0.8571428571428571), (51, 4.52), (210, 0.4), (112, 1.0508474576271187), (114, 1.5478260869565217), (200, 0.7189189189189189), (106, 1.7692307692307692), (334, 0.05723905723905724), (-9, 2.971014492753623), (113, 1.1419354838709677), (104, 1.0988593155893536), (92, 1.0261194029850746), (100, 1.1470588235294117), (165, 0.47560975609756095), (105, 1.2820512820512822), (229, 0.313953488372093), (73, 1.1275167785234899), (96, 1.1081081081081081), (144, 0.49206349206349204), (190, 0.1411764705882353), (105, 1.0497737556561086), (78, 0.9354838709677419), (111, 1.1533546325878594), (91, 1.3333333333333333), (46, 1.7833333333333334), (70, 1.9761904761904763), (90, 1.1145833333333333), (52, 1.3877551020408163), (209, 3.2), (91, 1.2820512820512822), (101, 1.2453987730061349), (95, 1.320754716981132), (94, 1.1923076923076923), (92, 1.091549295774648), (119, 1.0807453416149069), (125, 2.3333333333333335), (108, 0.6099290780141844), (123, 2.3488372093023258), (349, 0.02120141342756184), (100, 1.1622516556291391), (138, 0.9945652173913043), (97, 1.3297872340425532), (88, 0.45454545454545453), (86, 1.1111111111111112), (4, 3.4285714285714284), (176, 0.6601941747572816), (106, 1.2258064516129032), (88, 1.5151515151515151), (102, 1.2348484848484849), (50, 1.2727272727272727), (82, 1.0384615384615385), (111, 0.6363636363636364), (126, 1.26), (67, 0.42857142857142855), (-8, 2.0115606936416186), (104, 1.0098039215686274), (71, 1.0346820809248556), (96, 1.1), (253, 1.0210526315789474), (128, 0.9487179487179487), (196, 0.39402173913043476), (101, 1.0063694267515924), (156, 0.8518518518518519), (55, 1.297709923664122), (83, 1.1928251121076232), (109, 1.0044052863436124), (102, 0.9871794871794872), (230, 0.31155778894472363), (66, 1.2096774193548387), (-172, 71.0), (152, 0.7), (63, 0.9032258064516129), (250, 4.333333333333333), (100, 1.0), (57, 0.375), (467, 39.0), (158, 0.48737373737373735), (60, 1.1492537313432836), (91, 0.875), (93, 1.1363636363636365), (91, 1.236842105263158), (111, 1.475), (103, 1.0136986301369864), (103, 0.9310344827586207), (121, 2.0952380952380953), (87, 1.2641509433962264), (154, 0.4972677595628415), (-42, 5.666666666666667), (92, 0.5882352941176471), (96, 3.032258064516129), (-2, 2.6), (110, 2.2857142857142856), (142, 0.23809523809523808), (117, 1.5), (145, 1.8904109589041096), (131, 4.125), (58, 1.1801470588235294), (124, 1.8043478260869565), (-32, 7.083333333333333)]

#scaleInfoList = [(104, 1.127659574468085), (-121, 1.2), (3, 1.8926553672316384), (-174, 1.2352941176470589), (99, 0.06363636363636363), (-80, 1.8840579710144927), (-330, 2.5555555555555554), (34, 0.1694915254237288), (-251, 1.4782608695652173), (44, 0.48148148148148145), (60, 0.625), (43, 0.1568627450980392), (168, 0.7089552238805971), (220, 1.1649214659685865), (65, 0.7165775401069518), (-22, 0.6178861788617886), (-412, 3.5), (64, 0.625), (-1230, 7.478260869565218), (50, 0.625), (-381, 2.1052631578947367), (-11, 0.58), (68, 0.6), (-6, 0.2711864406779661), (10, 1.0), (-274, 2.125), (-3375, 16.0), (251, 0.875), (-413, 3.0), (78, 1.1538461538461537), (-966, 6.2745098039215685), (22, 0.0), (-27, 1.3157894736842106), (131, 0.901840490797546), (27, 0.24358974358974358), (14, 0.08571428571428572), (87, 0.56), (-232, 3.0144927536231885), (-417, 2.625), (-168, 1.0), (32, 0.2857142857142857), (-3390, 17.0), (-406, 2.0), (155, 0.5714285714285714), (-16, 0.5735294117647058), (-137, 3.5714285714285716), (21, 0.17670682730923695), (-210, 1.8571428571428572), (37, 0.20901639344262296), (12, 0.12264150943396226), (-616, 3.6666666666666665), (-131, 1.25), (-123, 1.5443037974683544), (149, 1.316831683168317), (37, 0.0), (7, 0.8358974358974359), (-104, 1.1354166666666667), (-114, 1.5294117647058822), (-148, 3.46875), (59, 0.0), (21, 0.43137254901960786), (-100, 1.393939393939394), (63, 0.5769230769230769), (-36, 0.9827586206896551), (-78, 1.4583333333333333), (-89, 0.8571428571428571), (236, 0.7058823529411765), (-154, 0.88), (49, 1.1578947368421053), (-179, 1.1176470588235294), (-3, 0.30357142857142855), (-11, 1.3859649122807018), (-87, 6.892857142857143), (-2, 0.8829787234042553), (154, 1.3668122270742358), (39, 1.6521739130434783), (23, 0.0), (-174, 1.7692307692307692), (-110, 1.4871794871794872), (-97, 1.0), (-93, 1.6194029850746268), (-120, 1.75), (73, 0.867816091954023), (-197, 2.0), (40, 0.36363636363636365), (-3306, 20.0), (115, 1.0810810810810811), (43, 0.2222222222222222), (-109, 1.6842105263157894), (21, 0.14285714285714285), (-212, 1.9375), (92, 1.2429906542056075), (-58, 1.0), (98, 0.38388625592417064), (100, 0.9629629629629629), (51, 1.619047619047619), (-103, 0.85), (-64, 0.6666666666666666), (-69, 0.6923076923076923), (-23, 1.0347826086956522), (-14, 4.380952380952381), (-186, 1.6), (-103, 0.7692307692307693), (69, 0.6137339055793991), (-216, 2.123456790123457), (10, 0.8488372093023255), (-199, 4.565656565656566), (38, 0.7265625), (16, 0.12152777777777778), (74, 1.2796610169491525), (-124, 1.0465116279069768), (112, 0.2149122807017544), (-54, 0.4), (-3469, 21.466666666666665), (73, 0.49122807017543857), (-123, 4.566666666666666), (39, 1.264367816091954), (-97, 1.6666666666666667), (-194, 1.5714285714285714), (84, 1.0555555555555556), (-19, 0.9666666666666667), (7, 0.08056872037914692), (-147, 1.0384615384615385), (-398, 4.235294117647059), (112, 2.5369127516778525), (-29, 1.0), (-10, 0.7011494252873564), (511, 6.0), (-93, 0.9705882352941176), (43, 2.8333333333333335), (-39, 0.5), (19, 0.1895734597156398), (11, 0.07667731629392971), (44, 0.4095238095238095), (64, 0.625), (64, 0.625), (-242, 3.564516129032258), (-28, 1.3333333333333333), (6, 0.6363636363636364), (-95, 1.0), (10, 0.5224719101123596), (-21, 0.8529411764705882), (-184, 0.8387096774193549), (6, 1.5087719298245614), (-69, 0.5379310344827586), (-50, 0.5384615384615384), (48, 0.625), (73, 1.0), (-67, 1.0), (132, 1.8571428571428572), (-53, 0.45161290322580644), (-146, 6.833333333333333), (-1135, 10.5), (48, 0.625), (1, 0.9117647058823529), (-23, 0.9928057553956835), (-167, 1.105263157894737), (25, 1.5211267605633803), (-147, 1.2222222222222223), (-10, 1.1206896551724137), (-4402, 28.0), (-4, 1.1627906976744187), (24, 0.6097560975609756), (-145, 1.2254901960784315), (14, 0.42857142857142855), (5, 1.0769230769230769), (-2384, 10.666666666666666), (-23, 1.490909090909091), (23, 0.676056338028169), (-208, 1.8), (-117, 1.0), (131, 0.07142857142857142), (78, 0.7766990291262136), (-191, 1.8333333333333333), (42, 1.2346938775510203), (-183, 1.8860759493670887), (83, 1.4180327868852458), (-75, 0.7777777777777778), (5, 0.06532663316582915), (26, 0.44), (117, 1.375586854460094), (-31, 0.36363636363636365), (-57, 1.7424242424242424), (5, 0.8666666666666667), (78, 0.7843137254901961), (64, 0.625), (-284, 3.24), (-142, 2.6), (-69, 0.7333333333333333), (-108, 1.457142857142857), (139, 1.487719298245614), (-84, 1.3571428571428572), (66, 0.6767676767676768), (-19, 1.0172413793103448), (-539, 4.166666666666667), (-41, 0.8333333333333334), (54, 0.3673469387755102), (121, 0.9568965517241379), (44, 0.0), (-26, 0.5), (107, 0.31313131313131315), (29, 0.544973544973545), (26, 1.3008849557522124), (49, 0.2962962962962963), (-231, 2.375), (-86, 1.6), (93, 0.5), (64, 0.625), (-73, 0.5789473684210527), (11, 0.75), (63, 0.6432160804020101), (123, 0.5869565217391305), (13, 0.1443850267379679), (24, 0.4647887323943662), (-4, 0.1875), (-182, 0.9905660377358491), (78, 0.5964912280701754), (-255, 2.6023391812865495), (-22, 1.1891891891891893), (-38, 1.1566265060240963), (-61, 2.0128205128205128), (-56, 1.0666666666666667), (-133, 2.7), (25, 0.2264957264957265), (-35, 0.8192771084337349), (-61, 2.581818181818182), (-359, 3.3333333333333335), (8, 1.0204081632653061), (208, 1.105740181268882), (16, 0.2692307692307692), (-443, 3.0), (87, 1.0056818181818181), (9, 0.4166666666666667), (-34, 1.1217948717948718), (77, 1.6896551724137931), (-139, 8.23076923076923), (0, 1.0), (-82, 1.0), (9, 0.13513513513513514), (-29, 1.4761904761904763), (14, 2.2222222222222223), (-70, 1.0731707317073171), (-45, 0.5), (-141, 1.1666666666666667), (88, 1.103658536585366), (-1452, 12.0), (-78, 1.25), (15, 0.15639810426540285), (-156, 1.3773584905660377), (209, 0.7884615384615384), (-13, 0.85), (158, 0.5507900677200903), (-30, 1.2159090909090908), (-142, 1.4285714285714286), (-70, 0.7894736842105263), (48, 0.625), (52, 0.029411764705882353), (43, 0.0), (-66, 1.875), (-138, 2.1025641025641026), (-36, 1.058139534883721), (-116, 2.75), (94, 0.46153846153846156), (-70, 0.6829268292682927), (-32, 0.7037037037037037), (84, 0.3643410852713178), (1, 0.63), (23, 1.4651162790697674), (55, 4.0), (-66, 0.9056603773584906), (-29, 0.5), (-15, 1.1067961165048543), (-538, 4.5), (-52, 0.4444444444444444), (35, 0.8333333333333334), (61, 0.9861111111111112), (-12807, 101.0), (90, 0.6438746438746439), (-33, 0.6), (305, 1.2125), (33, 0.7266666666666667), (-24, 0.9933333333333333), (-65, 0.85), (48, 0.8505747126436781), (127, 0.05263157894736842), (-48, 0.975609756097561), (39, 0.2857142857142857), (12, 0.1656441717791411), (45, 1.1904761904761905), (-46, 0.7368421052631579), (14, 0.2), (-346, 3.1076923076923078), (-23, 1.0), (61, 0.448), (-290, 3.4285714285714284), (25, 0.2602040816326531), (57, 1.1071428571428572), (-18, 1.2), (-144, 0.9310344827586207), (-5, 1.5142857142857142)]

In [ ]:
import random
import math
import time
import numpy as np

import os.path

clusterList = [cluster1, cluster2, cluster3, cluster4]
clusterList = [cluster1, cluster2, cluster3]
#clusterList = [cluster2, cluster3]

#objTypeList = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']
#scaleObjTypeList = [1.05, 1.0, 1.1, 1.2, 0.2, 0.2, 0.25]

#objTypeList = ['person', 'car', 'motorcycle', 'truck', 'bird', 'cat', 'dog']
#scaleObjTypeList = [1.0, 1.0, 0.9, 1.1, 0.22, 0.25, 0.28]

#objTypeList = ['dog', 'person', 'car', 'motorcycle', 'truck', 'bird', 'cat']
#scaleObjTypeList = [0.28, 1.0, 1.0, 0.9, 1.1, 0.22, 0.25]

#Removed cat, dog, bird
objTypeList = ['person', 'car', 'motorcycle', 'truck']
scaleObjTypeList = [1.0, 1.0, 0.9, 1.1]

#clustInd = -1

initTime = time.perf_counter()

print(initTime)

#for iterInd in range(5):
#for iterInd in [1]:
for iterInd in range(1):
    
    clustInd = -1
    
    iterNum = iterInd + 1

    for clusterNum in clusterList:

        clustInd = clustInd + 1

        if clustInd==0:

            clustMaxDict = maxDictGCP1
        elif clustInd==1:

            clustMaxDict = maxDictGCP2
        elif clustInd==2:

            clustMaxDict = maxDictGCP3
            
        '''
        elif clustInd==3:

            clustMaxDict = maxDictGCP4
        '''

        for objTypeNum in objTypeList:

            curObjTypInd = objTypeList.index(objTypeNum)
            curScaleVal = scaleObjTypeList[curObjTypInd]

            if objTypeNum=='person':

                obj1 = './pedestrianFemale.png'
                obj2 = './pedestrianMale.png'
                obj1Dim = (0, 1411, 0, 1045)
                obj2Dim = (0, 1023, 0, 568)

            elif objTypeNum=='car':

                obj1 = './Tesla.png'
                obj2 = './mini-car.png'
                obj1Dim = (1, 430, 0, 573)
                obj2Dim = (1, 853, 0, 832)

            elif objTypeNum=='motorcycle':

                obj1 = './bike1.png'
                obj2 = './bike2.png'
                obj1Dim = (13, 662, 0, 489)
                obj2Dim = (0, 1838, 0, 1331)

            elif objTypeNum=='truck':

                obj1 = './truck1.png'
                obj2 = './truck2.png'
                obj1Dim = (31, 595, 0, 748)
                obj2Dim = (13, 537, 0, 692)

            elif objTypeNum=='bird':

                obj1 = './bird.png'
                obj2 = './crow.png'
                obj1Dim = (1, 624, 0, 525)
                obj2Dim = (1, 723, 0, 673)

            elif objTypeNum=='cat':

                obj1 = './black-cat.png'
                obj2 = './cat.png'
                obj1Dim = (1, 1959, 0, 1678)
                obj2Dim = (16, 700, 0, 472)

            elif objTypeNum=='dog':

                obj1 = './runningDog.png'
                obj2 = './standingDog3.png'
                obj1Dim = (1, 354, 0, 694)
                obj2Dim = (1, 799, 0, 644)

            if objTypeNum in clustMaxDict:

                maxLimit = clustMaxDict[objTypeNum]
            else:

                maxLimit = 0

            minAdd = maxLimit+1

            if objTypeNum=='bird' or objTypeNum=='cat' or objTypeNum=='dog':

                minAdd = minAdd + 1

            for i in clusterNum:

                curDict = testList[i]

                if objTypeNum in curDict:

                    curNumObj = curDict[objTypeNum]

                else:

                    curNumObj = 0

                #addNumCount = minAdd - curNumObj
                
                if curNumObj<(maxLimit-1):
                    
                    addNumCount = 1
                else:
                    
                    addNumCount = 0
                    
                if (addNumCount==0):
                    
                    continue

                
                curNum = str(i).zfill(6)

                strName = "bremen_" + curNum + "_000019_leftImg8bit"

                strNameAdd = strName + ".png"

                openName = "./BremenCase/" + strNameAdd
                
                
                '''
                strName = "COCO" + str(i) 
                
                strNameAdd = strName + ".jpg"
    
                openName = "./Clean/MSCOCO/Raw/" + strNameAdd
                '''
        
                
                pathSkip = False
                        
                for pathNameInd in range(1):
                
                    fullPathName = ("CleanID/" + "Bremen/" + "AWS/" + "Insert/" + "Iter" + 
                                    str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                    str(addNumCount+pathNameInd) + ".png")

                    if os.path.exists(fullPathName):
                        
                        fileImage = scale_rotate_image(Image.open(fullPathName, 'r'), img_scale = 1, img_rotate = 0)
                        
                        fileImage.save("CleanID/" + "Bremen/" + "GCP/" + "Insert/" + "Iter" + 
                                         str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                         str(addNumCount+pathNameInd) + ".png")
                        
                        pathSkip = True
                        
                        break
                
                if (pathSkip):
                    
                    continue
                    
                for pathNameInd in range(1):
                
                    fullPathName = ("CleanID/" + "Bremen/" + "MS/" + "Insert/" + "Iter" + 
                                    str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                    str(addNumCount+pathNameInd) + ".png")

                    if os.path.exists(fullPathName):
                        
                        fileImage = scale_rotate_image(Image.open(fullPathName, 'r'), img_scale = 1, img_rotate = 0)
                        
                        fileImage.save("CleanID/" + "Bremen/" + "GCP/" + "Insert/" + "Iter" + 
                                         str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                         str(addNumCount+pathNameInd) + ".png")
                        
                        pathSkip = True
                        
                        break
                
                if (pathSkip):
                    
                    continue
                
                
                
                #Check for Duplicate
                '''
                pathSkip = False
                        
                for pathNameInd in range(10):
                
                    fullPathName = ("Clean/" + "MSCOCO/" + "MS/" + "Insert/" + "Iter" + 
                                    str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                    str(addNumCount+pathNameInd) + ".png")

                    if os.path.exists(fullPathName):
                        
                        fileImage = scale_rotate_image(Image.open(fullPathName, 'r'), img_scale = 1, img_rotate = 0)
                        
                        fileImage.save("Clean/" + "MSCOCO/" + "GCP/" + "Insert/" + "Iter" + 
                                         str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                         str(addNumCount+pathNameInd) + ".png")
                        
                        pathSkip = True
                        
                        break
                
                if (pathSkip):
                    
                    continue
                '''

                background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
                
                backWidth, backHeight = background1.size

                #background1 = background1.resize((2048, 1024))

                cfg = get_cfg()
                cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
                cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
                predictor = DefaultPredictor(cfg)

                curImg = np.array(background1.convert('RGB'))
                panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

                if objTypeNum=='car' or objTypeNum=='truck' or objTypeNum=='motorcycle':

                    baseIds = getRoadIDCar(segments_info)

                else:

                    baseIds = getRoadID(segments_info)

                newPanoArr = panoptic_seg.cpu().numpy()

                tryVals = 0
                tryList = []
                tryBoolList = []
                tryBoolAccomList = []

                tryCount = 0

                while(tryVals<30):

                    ranW = random.randint(0, backWidth-1)
                    ranH = random.randint(0, backHeight-1)

                    if newPanoArr[ranH][ranW] in baseIds:

                        tryList.append((ranH,ranW))
                        tryBoolList.append(False)
                        tryBoolAccomList.append(0)

                        tryVals = tryVals + 1

                    if tryCount>300:

                        break

                    tryCount = tryCount + 1

                tryList.sort()

                for tryLoopCount, itemTry in enumerate(tryList):

                    ranH, ranW = itemTry

                    scale50, scalingFac = scaleInfoList[i]

                    expectedHeight = math.floor(scale50 + (ranH-(0.5*backHeight))*scalingFac)

                    if expectedHeight<0.03*backHeight:

                        continue

                    if expectedHeight<0.05*backHeight:

                        #expectedHeight = 40
                        expectedHeight = math.floor(0.05*backHeight)

                    expectedHeight = math.floor(expectedHeight*curScaleVal)

                    chosenInsert = random.choice([1, 2])

                    if chosenInsert==1:

                        chosenPic = obj1
                        chosenDim = obj1Dim
                    else:

                        chosenPic = obj2
                        chosenDim = obj2Dim

                    top, bot, left, right = chosenDim

                    curHeight = (bot-top)

                    imgScaleFac = expectedHeight/curHeight

                    chosenImg = scale_rotate_image(Image.open(chosenPic, 'r'), 
                                                   img_scale = imgScaleFac, img_rotate = 0)

                    imWidth, imHeight = chosenImg.size

                    checkW1 = ranW
                    checkW2 = ranW + imWidth

                    checkH1 = ranH
                    checkH2 = ranH - math.floor(imHeight/2)

                    if checkW2>(backWidth-1) or ranH-imHeight<0:

                        break

                    else:

                        line1 = newPanoArr[checkH1][checkW1:checkW2]
                        line2 = newPanoArr[checkH2][checkW1:checkW2]

                        listCheck1 = []
                        listCheck2 = []

                        for elemID in baseIds:

                            checkBoolVal = line1==elemID
                            listCheck1.append(checkBoolVal)

                            checkBoolVal = line2==elemID
                            listCheck2.append(checkBoolVal)

                        oldVar1 = listCheck1[0]

                        for elemVar1 in listCheck1:

                            oldVar1 = oldVar1 | elemVar1

                        oldVar2 = listCheck2[0]

                        for elemVar2 in listCheck2:

                            oldVar2 = oldVar2 | elemVar2

                        check1 = all(oldVar1)
                        check2 = all(oldVar1)

                        if (check1 and check2):

                            tryBoolList[tryLoopCount] = True
                            tryBoolAccomList[tryLoopCount] = (chosenInsert, imgScaleFac)

                if sum(tryBoolList)>=addNumCount:

                    indBoolList = np.where(tryBoolList)[0]
                    indBoolList = indBoolList.tolist()

                    found = False
                    whileCount = -1

                    finChoices = []

                    while (found==False):

                        whileCount = whileCount + 1

                        if whileCount>100:

                            break
                            
                        try:

                            curChoices = random.sample(indBoolList, addNumCount)
                            curChoices.sort()
                        except:
                            
                            print(strName)
                            break

                        curPanoArr = np.copy(newPanoArr)
                        objAddCheckerCount = 0

                        for elemChoice in curChoices:

                            tryCord = tryList[elemChoice]
                            tryInsert1, tryInsertScale1 = tryBoolAccomList[elemChoice]

                            if tryInsert1==1:

                                chosenPic = obj1
                                chosenDim = obj1Dim
                            else:

                                chosenPic = obj2
                                chosenDim = obj2Dim

                            chosenImg = scale_rotate_image(Image.open(chosenPic, 'r'), 
                                                           img_scale = tryInsertScale1, img_rotate = 0)

                            imWidth, imHeight = chosenImg.size

                            tryH, tryW = tryCord

                            checkW1 = tryW
                            checkW2 = tryW + imWidth

                            checkH1 = tryH
                            checkH2 = tryH - math.floor(imHeight/2)

                            if checkW2>(backWidth-1) or ranH-imHeight<0:

                                break

                            else:

                                line1 = curPanoArr[checkH1][checkW1:checkW2]
                                line2 = curPanoArr[checkH2][checkW1:checkW2]

                                if (1000 in line1) or (1000 in line2):

                                    break
                                else:

                                    foundW = tryW
                                    foundH = tryH - imHeight

                            for foundIndsH in range(foundH, checkH1):

                                curPanoArr[foundIndsH][checkW1:checkW2] = 1000

                            objAddCheckerCount = objAddCheckerCount + 1

                            if (objAddCheckerCount==addNumCount):

                                finChoices = curChoices
                                found = True
                    if found:

                        for elemChoice in curChoices:

                            tryCord = tryList[elemChoice]
                            tryInsert1, tryInsertScale1 = tryBoolAccomList[elemChoice]

                            if tryInsert1==1:

                                chosenPic = obj1
                                chosenDim = obj1Dim
                            else:

                                chosenPic = obj2
                                chosenDim = obj2Dim

                            chosenImg = scale_rotate_image(Image.open(chosenPic, 'r'), 
                                                           img_scale = tryInsertScale1, img_rotate = 0)

                            imWidth, imHeight = chosenImg.size

                            tryH, tryW = tryCord

                            foundW = tryW
                            foundH = tryH - imHeight

                            background1.paste(chosenImg, (foundW, foundH), mask=chosenImg)

                    if found:

                        
                        background1.save("CleanID/" + "Bremen/" + "GCP/" + "Insert/" + "Iter" + 
                                         str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                         str(addNumCount) + ".png")
                        
                        '''
                        background1.save("CleanID/" + "MSCOCO/" + "GCP/" + "Insert/" + "Iter" + 
                                         str(iterNum) + "/" + objTypeNum + "/" + strName + "Count" + 
                                         str(addNumCount) + ".png")
                        '''

finTime = time.perf_counter()
elapTime = finTime - initTime
print(elapTime)

In [ ]:
curNum = str(4).zfill(6)
    
strName = "bremen_" + curNum + "_000019_leftImg8bit.png"

openName = "./BremenCase/" + strName

background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)

background1 = background1.resize((2048, 1024))

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)

curImg = np.array(background1.convert('RGB'))
panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

print(panoptic_seg)

newArrayTest = panoptic_seg.cpu().numpy()

for i in [0, 1]:
    
    newArrayTest[i][0:2] = 1000

#panoptic_seg[0:1][0] = 100

print(newArrayTest)

In [ ]:
import time
import math

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']

#Person - 1
#Car - 2
#MotorCycle - 3
#Truck - 4

classType = 'truck'
strVal = str(3+1).zfill(3)

#imgBlack = Image.new('RGB', (2048, 1024), color = 'black')
    
#imgWhite = Image.new('RGB', (2048, 1024), color = 'white')

initTime = time.perf_counter()

print(initTime)

for i in range(300):
    
    curNum = str(i).zfill(6)
    
    '''
    strName = "bremen_" + curNum + "_000019_leftImg8bit"
    
    strNameAdd = strName + ".png"
    
    openName = "./BremenCase/" + strNameAdd
    '''
    
    strName = "COCO" + str(i) 
                
    strNameAdd = strName + ".jpg"
    
    openName = "./Clean/MSCOCO/Raw/" + strNameAdd
    
    background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)
    
    #background1 = background1.resize((2048, 1024))
    
    imWidth, imHeight = background1.size
    
    imgBlack = Image.new('RGB', (imWidth, imHeight), color = 'black')
    
    imgWhite = Image.new('RGB', (imWidth, imHeight), color = 'white')
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    predictor = DefaultPredictor(cfg)

    curImg = np.array(background1.convert('RGB'))
    panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]
    
    segIds = getSegmentID(segments_info, classType)
    
    boolArr = panoptic_seg.cpu().numpy()
    
    listCheck1 = []
    oldVar1 = []
    
    for segID in segIds:

        checkBoolVal = boolArr==segID
        listCheck1.append(checkBoolVal)
        
    if len(listCheck1)>0:
        
        oldVar1 = listCheck1[0]

        for elemVar1 in listCheck1:

            oldVar1 = oldVar1 | elemVar1
            
        '''
        rows, cols = np.where(oldVar1)
        
        rowMin = min(rows)
        rowMax = max(rows)
        colMin = min(cols)
        colMax = max(cols)
        
        rowMid = math.floor((rowMax+rowMin)/2)
        colMid = math.floor((colMax+colMin)/2)
        
        print(rowMid, colMid)
        '''
            
        mask = np.expand_dims(oldVar1, axis=-1)
        mask2 = np.concatenate((mask, mask, mask), axis=-1)
        
        crops = curImg * mask2
        
        rgba = cv2.cvtColor(crops, cv2.COLOR_RGB2RGBA)
        rgba[:, :, 3] = mask[:, :, 0]*255
            
        boolImg = Image.fromarray(oldVar1)
        
        im = Image.composite(imgWhite, imgBlack, boolImg)
        
        imRGB = im.convert('RGB')
        open_cv_image = np.array(imRGB)
        open_cv_image = open_cv_image[:, :, ::-1].copy()
        
        kernel = np.ones((11,11),np.uint8)
        dilation = cv2.dilate(open_cv_image,kernel,iterations = 3)
        
        #strVal = str(0+1).zfill(3)
        
        strImgMask = "./COCODelete/" + strName + "_mask" + strVal + ".png"
        
        cv2.imwrite(strImgMask, dilation)
        #dilation.save(strImgMask)
        
#print("Done")

finTime = time.perf_counter()
elapTime = finTime - initTime

print(elapTime)

In [ ]:
from scipy import ndimage

struct2 = ndimage.generate_binary_structure(2, 2)

def getSegmentIDSingleFull(panoptic_seg, segments_info, classType):

    import json
    with open('Categories.txt') as f:
        catList = json.load(f)

    itemIDs = []

    for item in segments_info:

        itemInd = item['category_id']

        isThingCheck = item['isthing']

        if not isThingCheck:

            itemInd = itemInd + 80
            
            itemID = item['id']
            
            panoptic_seg[panoptic_seg==itemID] = 1000

        curType = catList[itemInd]['name']

        if curType == classType:

            itemID = item['id']
            itemIDs.append(itemID)
            
    chosenId = []
    
    notFound = True
    
    if(itemIDs):
        
        while notFound:
            
            if (itemIDs):
        
                randomChoice = random.choice(itemIDs)
                #chosenId.append(randomChoice)

                checkBoolVal = newPanoArr==randomChoice
                rows, cols = np.where(checkBoolVal)

                curMin = min(rows)
                curMax = max(rows)
                
                if (curMax-curMin)<50:
                    
                    itemIDs.remove(randomChoice)
                    continue

                boolArr = panoptic_seg==randomChoice

                boolArrEdge = ndimage.binary_dilation(boolArr, structure=struct2, iterations=5).astype(boolArr.dtype)

                edgeArr = np.logical_xor(boolArr, boolArrEdge)

                objIndexed = panoptic_seg[edgeArr]

                objIndexed = np.unique(objIndexed)
                                
                if 0 in objIndexed:
                    
                    objIndexed = objIndexed[1:]
                    
                if 1000 in objIndexed:

                    objIndexed = objIndexed[:-1]
                    
                rowMaxFin = 0
                
                if (objIndexed.size != 0):

                    for elemID in objIndexed:

                        checkBoolVal = newPanoArr==elemID
                        rows, cols = np.where(checkBoolVal)

                        rowMax = max(rows)

                        if rowMax>rowMaxFin:

                            rowMaxFin = rowMax

                    if curMax>rowMaxFin:

                        notFound = False
                        chosenId.append(randomChoice)
                    else:

                        itemIDs.remove(randomChoice)
                else:
                    
                    notFound = False
                    chosenId.append(randomChoice)
                    
            else:
                
                break
    
    return chosenId

In [ ]:
import time
import math

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']
classes = ['person', 'car', 'motorcycle', 'truck']

classType = 'car'

#imgBlack = Image.new('RGB', (2048, 1024), color = 'black')
    
#imgWhite = Image.new('RGB', (2048, 1024), color = 'white')

fullList = []

firstTime = time.perf_counter()

for iterationNum in [2, 3, 4, 5]:

    printList = []

    for classIndex, elemClass in enumerate(classes):

        print(elemClass)
        classCoordList = []

        timeNow = time.perf_counter()

        for i in range(300):

            #curNum = str(i).zfill(6)

            '''
            strName = "bremen_" + curNum + "_000019_leftImg8bit"

            strNameAdd = strName + ".png"

            openName = "./BremenCase/" + strNameAdd
            '''

            
            strName = "COCO" + str(i) 

            strNameAdd = strName + ".jpg"

            openName = "./Clean/MSCOCO/Raw/" + strNameAdd
            

            background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)

            #background1 = background1.resize((2048, 1024))

            imWidth, imHeight = background1.size

            imgBlack = Image.new('RGB', (imWidth, imHeight), color = 'black')

            imgWhite = Image.new('RGB', (imWidth, imHeight), color = 'white')

            cfg = get_cfg()
            cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
            cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
            predictor = DefaultPredictor(cfg)

            curImg = np.array(background1.convert('RGB'))
            panoptic_seg, segments_info = predictor(curImg)["panoptic_seg"]

            newPanoArr = panoptic_seg.cpu().numpy()

            segIds = getSegmentIDSingleFull(newPanoArr, segments_info, elemClass)

            listCheck1 = []
            oldVar1 = []

            for segID in segIds:

                checkBoolVal = newPanoArr==segID
                listCheck1.append(checkBoolVal)

            if len(listCheck1)>0:

                oldVar1 = listCheck1[0]

                for elemVar1 in listCheck1:

                    oldVar1 = oldVar1 | elemVar1

                rows, cols = np.where(oldVar1)

                rowMin = min(rows)
                rowMax = max(rows)
                colMin = min(cols)
                colMax = max(cols)

                rowMid = math.floor((rowMax+rowMin)/2)
                colMid = math.floor((colMax+colMin)/2)

                #print(rowMid, colMid)
                classCoordList.append((rowMid, colMid))

                mask = np.expand_dims(oldVar1, axis=-1)
                mask2 = np.concatenate((mask, mask, mask), axis=-1)

                crops = curImg * mask2

                rgba = cv2.cvtColor(crops, cv2.COLOR_RGB2RGBA)
                rgba[:, :, 3] = mask[:, :, 0]*255

                imPIL = Image.fromarray(rgba)

                #imRGB = rgba.convert('RGB')
                #open_cv_image = np.array(imRGB)
                #open_cv_image = open_cv_image[:, :, ::-1].copy()

                strVal = str(classIndex+1).zfill(3)
                
                strImgMask = ("./Rotate/Iter" + str(iterationNum) + "/Iter" + 
                              str(iterationNum) + "COCORotateImages/" + 
                              strName + "_mask" + strVal + ".png")

                #strImgMask = "./Rotate/Iter2/Iter2BremenRotateImages/" + strName + "_mask" + strVal + ".png"

                #cv2.imwrite(strImgMask, rgba)
                imPIL.save(strImgMask)

                boolImg = Image.fromarray(oldVar1)

                im = Image.composite(imgWhite, imgBlack, boolImg)

                imRGB = im.convert('RGB')
                open_cv_image = np.array(imRGB)
                open_cv_image = open_cv_image[:, :, ::-1].copy()

                '''
                strImgMask = "./TestOOD/" + strName + "_mask" + strVal + ".png"

                cv2.imwrite(strImgMask, open_cv_image)

                temp = Image.open(strImgMask)

                plt.figure()
                plt.imshow(temp)
                '''

                kernel = np.ones((11,11),np.uint8)
                dilation = cv2.dilate(open_cv_image,kernel,iterations = 3)

                strVal = str(classIndex+1).zfill(3)
                
                strImgMask = ("./Rotate/Iter" + str(iterationNum) + "/Iter" + 
                              str(iterationNum) + "COCORotateMask/" + 
                              strName + "_mask" + strVal + ".png")

                #strImgMask = "./Rotate/Iter2/Iter2BremenRotateMask/" + strName + "_mask" + strVal + ".png"

                cv2.imwrite(strImgMask, dilation)
                #dilation.save(strImgMask)

                '''
                temp = Image.open(strImgMask)

                plt.figure()
                plt.imshow(temp)
                '''

        timeEnd = time.perf_counter()
        elapTime = timeEnd - timeNow
        print(elapTime)
        print(classCoordList)

        printList.append(elemClass)
        printList.append(classCoordList)

    print("Done")
    print(iterationNum)
    print(printList)
    print("Act Done")
                              
    fullList.append(printList)

print("Full List")
print(fullList)
print("Time")
timeEnd2 = time.perf_counter()
elapTime = timeEnd2 - firstTime
print(elapTime)
    

In [ ]:
#Iter 1
'''
personCoordList = [(468, 125), (440, 1694), (434, 759), (445, 1605), (432, 1577), (475, 2022), (420, 1213), (494, 59), (427, 162), (436, 1675), (480, 183), (414, 688), (456, 1415), (424, 1306), (430, 1100), (498, 1350), (467, 602), (495, 423), (481, 662), (419, 735), (429, 588), (402, 1692), (408, 622), (422, 489), (472, 534), (462, 1065), (460, 1818), (463, 1033), (466, 739), (466, 1824), (374, 1207), (471, 168), (446, 1047), (440, 292), (512, 89), (456, 490), (469, 407), (473, 1176), (438, 1527), (500, 824), (450, 1313), (464, 1075), (468, 1343), (489, 1668), (452, 553), (449, 1445), (438, 1086), (438, 1833), (482, 536), (448, 638), (448, 1727), (539, 487), (451, 1658), (434, 1117), (485, 924), (475, 1164), (441, 1511), (332, 916), (457, 482), (401, 1810), (391, 448), (447, 1792), (412, 1293), (484, 107), (445, 1565), (483, 1755), (463, 1470), (480, 1841), (457, 804), (544, 1605), (465, 704), (461, 905), (434, 1278), (448, 1381), (421, 1913), (455, 1686), (442, 987), (435, 511), (445, 267), (453, 1374), (454, 558), (447, 924), (439, 1315), (417, 1059), (425, 1396), (425, 1545), (450, 2034), (450, 558), (423, 1520), (421, 76), (405, 1141), (577, 988), (465, 1423), (439, 666), (475, 218), (429, 804), (464, 260), (446, 170), (436, 1492), (247, 615), (389, 1415), (450, 1660), (453, 372), (453, 1471), (450, 264), (477, 17), (436, 1126), (484, 1144), (464, 603), (444, 1237), (478, 1765), (453, 1664), (473, 858), (446, 612), (418, 1360), (453, 1577), (453, 1105), (475, 1031), (459, 783), (441, 1149), (485, 484), (393, 1655), (368, 1775), (454, 1655), (480, 1347), (450, 1126), (447, 705), (463, 33), (426, 1250), (447, 1236), (479, 331), (460, 1117), (457, 1454), (460, 1457), (494, 1545), (427, 1470), (457, 1319), (452, 397), (436, 1263), (368, 285), (442, 1322), (513, 388), (436, 1250), (469, 973), (460, 40), (480, 204), (431, 1620), (489, 166), (442, 374), (447, 711), (466, 1264), (430, 1456), (424, 1983), (438, 1844), (482, 324), (468, 1560), (488, 430), (427, 1869), (443, 1278), (500, 262), (436, 1546), (441, 1926), (463, 120), (439, 1496), (453, 1360), (413, 1474)]

carCoordList = [(409, 1651), (567, 361), (635, 1207), (498, 948), (485, 1098), (468, 1163), (660, 1811), (650, 916), (489, 1013), (495, 935), (461, 1412), (687, 1969), (475, 506), (433, 851), (581, 1863), (485, 767), (450, 1175), (472, 925), (466, 1549), (534, 869), (457, 980), (415, 550), (446, 52), (488, 412), (460, 834), (480, 768), (649, 1023), (471, 580), (485, 1150), (609, 1868), (487, 795), (578, 1269), (565, 318), (469, 1092), (369, 575), (471, 1762), (447, 1209), (431, 561), (580, 1852), (452, 1259), (417, 1670), (501, 578), (496, 914), (531, 873), (477, 1015), (736, 1982), (454, 739), (479, 939), (461, 1911), (431, 959), (524, 400), (719, 142), (446, 194), (473, 393), (533, 198), (605, 848), (465, 836), (395, 1871), (469, 1460), (534, 740), (530, 688), (471, 1196), (469, 20), (496, 1028), (460, 1384), (567, 159), (536, 1790), (422, 1123), (485, 699), (541, 176), (633, 126), (456, 904), (462, 660), (453, 1099), (539, 207), (448, 871), (462, 1094), (463, 889), (497, 60), (471, 956), (490, 1021), (451, 1340), (437, 908), (513, 198), (656, 1800), (578, 1797), (488, 1393), (473, 717), (491, 693), (511, 992), (490, 1336), (455, 814), (547, 463), (446, 1424), (466, 639), (411, 1982), (498, 793), (510, 1500), (618, 1757), (561, 280), (537, 471), (449, 838), (468, 1633), (459, 1158), (471, 2002), (473, 1396), (491, 668), (463, 1018), (489, 978), (460, 1383), (482, 72), (462, 756), (449, 363), (481, 63), (492, 150), (470, 1799), (460, 1769), (459, 1178), (468, 1301), (483, 540), (466, 862), (474, 1067), (545, 326), (478, 1131), (444, 859), (463, 754), (470, 797), (486, 1445), (453, 1774), (535, 446), (622, 270), (515, 933), (473, 1190), (455, 1382), (466, 1159), (519, 1179), (539, 508), (496, 1642), (451, 596), (490, 953), (467, 283), (464, 517), (470, 1050), (486, 133), (468, 1004), (519, 103), (459, 653), (531, 1517), (410, 64), (503, 1413), (383, 1078), (477, 1024), (458, 716), (497, 1363), (466, 44), (464, 544), (584, 1924), (458, 1200), (482, 55), (588, 1895), (458, 1401), (530, 1312), (498, 1520), (460, 860), (491, 596), (450, 126), (645, 29), (440, 594), (480, 1511), (603, 1905), (611, 88), (487, 1961), (460, 907), (560, 1840), (612, 434), (584, 30), (494, 1435), (445, 1095), (522, 1556), (525, 1515), (480, 649), (577, 1597), (684, 230), (564, 1895), (583, 148), (474, 746), (463, 1312), (564, 281), (458, 738), (584, 1835), (591, 1813), (628, 225), (475, 866), (448, 1373), (511, 1559), (690, 1961), (448, 1221), (498, 650), (542, 122), (445, 1086), (464, 1085), (532, 202), (440, 1096), (458, 967), (575, 1841), (461, 1062), (493, 1185), (543, 152), (543, 126), (594, 1873), (464, 473), (582, 1851), (517, 1798), (465, 1290), (535, 1795), (501, 63), (474, 910), (461, 914), (528, 1560), (515, 337), (545, 459), (465, 1135), (438, 878), (572, 1300), (474, 1081), (467, 1090), (426, 1311), (470, 884), (427, 1391), (551, 2026), (522, 65), (415, 448), (456, 1166), (458, 575), (424, 367), (556, 389), (542, 289), (483, 1772), (715, 1971), (441, 832), (444, 1549), (548, 75), (535, 1875), (492, 1110), (507, 211), (567, 286), (460, 1102), (601, 1845), (460, 790), (516, 943), (577, 89), (445, 323), (495, 367), (470, 547), (445, 122), (459, 1060), (589, 1786), (455, 1298), (478, 1449), (472, 812), (452, 1330), (611, 183), (479, 237), (444, 1471), (472, 787), (486, 1136), (474, 1104), (458, 873), (482, 1088), (496, 990), (464, 48), (480, 1021), (455, 645), (442, 1244), (475, 744), (448, 331)]

motorcycleCoordList = [(520, 1415), (509, 1050), (527, 1589), (545, 1435), (471, 233), (496, 1219), (556, 217), (611, 1759), (470, 1356), (484, 1301), (503, 1428), (510, 1371), (520, 524), (503, 1660), (485, 498), (510, 239), (495, 985), (479, 1797), (443, 1984)]

truckCoordList = [(412, 1374), (490, 145), (393, 1397), (431, 1360), (451, 688), (419, 594), (439, 1538), (414, 438), (458, 671), (454, 1130), (485, 163), (435, 941), (432, 1251), (446, 306), (437, 855), (393, 1711), (409, 1209), (376, 1835), (420, 565), (438, 745), (430, 1598), (455, 778), (418, 443), (427, 951), (424, 482), (439, 858), (418, 1128), (522, 1782), (439, 1023), (379, 1397), (428, 195), (415, 968), (383, 517), (417, 369), (449, 1864), (485, 1807), (629, 159), (435, 1118), (377, 55)]


personCoordListCOCO = [(119, 31), (415, 232), (255, 371), (356, 220), (225, 121), (311, 229), (165, 534), (186, 560), (388, 354), (373, 68), (392, 528), (171, 7), (197, 156), (244, 333), (285, 545), (423, 485), (316, 584), (436, 314), (340, 102), (249, 32), (358, 380), (466, 190), (145, 231), (423, 203), (313, 19), (247, 80), (315, 621), (368, 69), (364, 302), (575, 394), (296, 475), (229, 571), (273, 68), (445, 439), (285, 145), (310, 77), (450, 394), (413, 173), (330, 241), (329, 146), (280, 279), (315, 71), (353, 488), (479, 281), (288, 64), (376, 148), (317, 581), (402, 51), (346, 310), (205, 482), (160, 624), (255, 412), (447, 59), (179, 369), (391, 224), (435, 321), (344, 397), (407, 605), (364, 590), (254, 355), (592, 330), (323, 466), (192, 83), (188, 554), (269, 7), (517, 351), (160, 24), (343, 594), (243, 444), (397, 6), (414, 594), (410, 340), (414, 548), (214, 12), (352, 72), (229, 257), (212, 251), (473, 441), (209, 556), (282, 532), (314, 419), (439, 12), (233, 9), (531, 80), (206, 572), (266, 575), (352, 262), (345, 432), (261, 181), (330, 621), (552, 92), (235, 309), (268, 74), (410, 32), (254, 431), (86, 624), (285, 239), (234, 319), (419, 218), (355, 588), (354, 6), (196, 79), (291, 97), (114, 225), (332, 120), (231, 560), (432, 292), (311, 80), (315, 41), (485, 266), (304, 25), (264, 529), (399, 22)]

carCoordListCOCO = [(358, 127), (376, 586), (257, 619), (347, 471), (295, 603), (359, 410), (253, 54), (163, 422), (242, 89), (418, 293), (253, 510), (459, 257), (371, 621), (169, 456), (327, 621), (406, 28), (454, 110), (180, 614), (411, 573), (332, 327), (385, 68), (139, 535), (357, 490), (415, 61), (448, 309), (308, 610), (371, 488), (574, 145), (453, 40), (345, 257), (233, 310), (325, 577), (418, 169), (247, 60), (326, 576), (329, 400), (328, 82), (329, 96), (381, 20), (362, 14), (302, 388), (288, 460), (511, 164), (231, 347), (380, 433), (351, 582), (272, 166), (305, 502), (237, 52), (251, 442), (408, 84), (332, 578), (437, 511), (419, 15), (557, 215), (253, 579), (313, 590), (592, 113), (600, 401), (264, 377), (236, 41), (202, 469), (192, 91), (428, 328), (274, 23), (304, 633), (303, 362), (439, 592), (197, 234), (380, 216), (400, 572), (249, 24), (216, 454), (221, 61), (386, 597), (326, 50), (258, 399), (428, 385), (427, 31), (280, 316), (196, 386), (97, 528), (213, 70), (255, 114), (315, 184), (526, 221), (344, 519), (439, 602), (443, 210), (461, 418), (234, 146), (159, 59), (369, 100), (325, 338), (144, 83), (399, 26), (226, 141), (297, 28), (292, 19), (375, 322), (298, 252), (129, 369), (360, 423), (319, 194), (307, 465), (259, 47), (445, 477), (299, 134), (281, 69), (207, 348), (225, 358), (94, 390), (198, 356), (496, 403), (180, 12), (382, 185), (381, 35), (429, 96), (317, 609), (376, 628), (175, 79), (438, 98), (341, 397)]

motorcycleCoordListCOCO = [(384, 347), (386, 44), (222, 213), (300, 217), (231, 433), (285, 414), (256, 302), (325, 471), (273, 400), (287, 372), (314, 346)]

truckCoordListCOCO = [(58, 326), (504, 371), (272, 351), (456, 496), (306, 621), (281, 474), (336, 330), (437, 85), (221, 351), (300, 230), (235, 457), (328, 302), (222, 71), (580, 278), (213, 578), (301, 132), (235, 357), (194, 306), (254, 19), (249, 195), (240, 344), (291, 464), (220, 368), (112, 522), (239, 304), (316, 328), (385, 432), (156, 28), (256, 355), (308, 320), (223, 369), (395, 200), (228, 125), (316, 15), (443, 219), (287, 156), (215, 137), (218, 343)]
'''


In [ ]:
#Iter 2

'''
personCoordList = [(468, 125), (447, 1588), (432, 908), (445, 1605), (457, 554), (475, 2022), (420, 1213), (450, 205), (427, 162), (408, 1939), (471, 1393), (414, 688), (456, 1415), (445, 986), (430, 1100), (497, 1444), (467, 602), (453, 1324), (481, 662), (419, 735), (421, 140), (402, 1692), (447, 1874), (422, 489), (472, 534), (444, 346), (473, 824), (465, 1283), (466, 739), (466, 1824), (364, 1554), (444, 1496), (389, 442), (440, 292), (510, 199), (456, 490), (449, 1217), (473, 1176), (438, 1527), (500, 824), (466, 1389), (464, 1075), (468, 1343), (485, 1962), (452, 553), (444, 1368), (438, 1086), (438, 1833), (482, 536), (432, 1280), (448, 1727), (539, 487), (451, 1658), (439, 1033), (485, 924), (475, 1164), (433, 1577), (332, 916), (496, 1430), (401, 1810), (473, 328), (433, 1631), (412, 1293), (484, 107), (445, 1565), (483, 1755), (471, 1662), (480, 1841), (461, 1142), (470, 1202), (456, 598), (461, 905), (434, 1278), (448, 1381), (421, 1913), (455, 1686), (442, 987), (435, 511), (445, 267), (453, 1374), (456, 481), (447, 924), (439, 1315), (417, 1059), (425, 1396), (425, 1545), (450, 2034), (450, 558), (438, 1339), (437, 676), (405, 1141), (577, 988), (465, 1423), (434, 365), (475, 218), (429, 804), (461, 300), (454, 245), (436, 1492), (247, 615), (389, 1415), (450, 1660), (453, 372), (453, 1471), (450, 264), (477, 17), (436, 1126), (484, 1144), (464, 603), (444, 1237), (478, 1765), (453, 1664), (459, 895), (446, 612), (434, 408), (466, 2008), (453, 1105), (471, 92), (459, 783), (435, 1348), (485, 484), (393, 1655), (368, 1775), (454, 1655), (480, 1347), (447, 1099), (447, 705), (439, 638), (421, 1270), (447, 1236), (479, 331), (459, 1026), (457, 1454), (452, 1610), (441, 1049), (437, 1425), (457, 14), (452, 397), (436, 1263), (368, 285), (442, 1322), (513, 388), (436, 1250), (469, 973), (460, 40), (480, 204), (431, 1620), (489, 166), (450, 1151), (436, 353), (468, 1198), (430, 1456), (424, 1983), (432, 1975), (482, 324), (476, 1481), (475, 376), (472, 42), (443, 1278), (501, 314), (436, 1546), (441, 1926), (461, 15), (439, 1496), (453, 1360), (450, 467)]

carCoordList = [(445, 1163), (567, 361), (635, 1207), (460, 1298), (465, 1381), (468, 1163), (758, 131), (439, 425), (489, 1013), (456, 1435), (461, 1412), (687, 1969), (457, 894), (433, 851), (581, 1863), (485, 767), (450, 1175), (626, 9), (513, 754), (536, 1454), (457, 980), (415, 550), (446, 52), (622, 1807), (460, 834), (487, 22), (649, 1023), (471, 580), (485, 1150), (609, 1868), (487, 795), (578, 1269), (492, 894), (469, 1092), (369, 575), (471, 1762), (447, 1209), (431, 561), (490, 1431), (452, 1259), (417, 1670), (503, 1524), (496, 914), (551, 94), (620, 1893), (469, 721), (466, 1017), (479, 939), (461, 1911), (431, 959), (524, 400), (719, 142), (446, 194), (473, 393), (533, 198), (463, 437), (427, 64), (395, 1871), (469, 1460), (614, 1291), (575, 113), (463, 1316), (469, 20), (496, 1028), (467, 1095), (567, 159), (487, 767), (422, 1123), (485, 699), (541, 176), (507, 1561), (457, 1103), (459, 1058), (456, 1289), (459, 1179), (448, 871), (462, 1094), (463, 889), (497, 60), (481, 1694), (428, 617), (451, 1340), (437, 908), (513, 198), (669, 218), (672, 178), (687, 1881), (487, 1068), (477, 1219), (488, 728), (484, 1841), (452, 25), (451, 960), (446, 1424), (466, 639), (458, 688), (495, 1204), (567, 167), (617, 278), (561, 280), (537, 471), (449, 838), (468, 1633), (459, 1158), (525, 676), (559, 1801), (491, 668), (463, 1018), (489, 978), (460, 1383), (469, 765), (462, 756), (466, 123), (481, 63), (461, 431), (471, 760), (460, 1769), (459, 1178), (468, 1301), (483, 540), (490, 1076), (474, 1067), (475, 794), (478, 1131), (481, 1435), (463, 754), (470, 797), (486, 1445), (453, 1774), (535, 446), (571, 1691), (515, 933), (473, 1190), (455, 1382), (466, 1159), (643, 94), (539, 508), (496, 1642), (459, 413), (490, 953), (474, 89), (457, 1066), (470, 1050), (473, 472), (468, 1004), (519, 103), (476, 1053), (479, 188), (488, 1572), (485, 1053), (383, 1078), (519, 1388), (464, 587), (497, 1363), (466, 44), (464, 544), (584, 1924), (502, 1812), (482, 55), (478, 1324), (482, 1597), (530, 1312), (498, 1520), (460, 860), (491, 596), (444, 1171), (645, 29), (440, 594), (480, 1511), (462, 915), (611, 88), (514, 844), (577, 42), (560, 1840), (481, 1470), (478, 517), (632, 1851), (445, 1095), (480, 664), (525, 1515), (480, 649), (692, 241), (561, 1495), (517, 507), (583, 148), (474, 746), (540, 1816), (564, 281), (526, 1830), (584, 1835), (506, 659), (639, 1843), (491, 1502), (592, 1938), (463, 841), (451, 1198), (448, 1221), (498, 650), (542, 122), (477, 803), (456, 1866), (469, 735), (440, 1096), (488, 545), (575, 1841), (461, 1062), (493, 1185), (543, 152), (569, 1858), (594, 1873), (464, 473), (460, 1275), (487, 491), (456, 890), (535, 1795), (532, 1787), (474, 910), (461, 914), (458, 720), (515, 337), (545, 459), (471, 962), (442, 630), (572, 1300), (445, 1425), (467, 1090), (426, 1311), (470, 884), (493, 1080), (498, 1284), (488, 1945), (415, 448), (540, 1921), (458, 575), (424, 367), (450, 1058), (542, 289), (483, 1772), (336, 628), (451, 710), (444, 1549), (486, 1055), (466, 1496), (526, 1439), (557, 1628), (567, 286), (552, 1885), (511, 232), (460, 790), (516, 943), (577, 89), (450, 725), (495, 367), (470, 547), (445, 122), (442, 1969), (589, 1786), (484, 792), (527, 87), (472, 812), (452, 1330), (611, 183), (462, 927), (480, 1919), (478, 1070), (486, 1136), (474, 1104), (458, 873), (482, 1088), (468, 1968), (464, 48), (462, 778), (461, 188), (442, 1244), (507, 204), (429, 754)]

motorcycleCoordList = [(520, 1415), (509, 1050), (527, 1589), (545, 1435), (471, 233), (496, 1219), (556, 217), (611, 1759), (470, 1356), (484, 1301), (503, 1428), (510, 1371), (520, 524), (503, 1660), (485, 498), (510, 239), (495, 985), (479, 1797), (443, 1984)]

truckCoordList = [(412, 1374), (490, 145), (393, 1397), (431, 1360), (451, 688), (419, 594), (439, 1538), (414, 438), (458, 671), (454, 1130), (485, 163), (435, 941), (432, 1251), (446, 306), (437, 855), (393, 1711), (409, 1209), (376, 1835), (420, 565), (438, 745), (430, 1598), (455, 778), (418, 443), (427, 951), (424, 482), (445, 1171), (418, 1128), (522, 1782), (439, 1023), (379, 1397), (430, 1610), (427, 675), (383, 517), (417, 369), (449, 1864), (485, 1807), (629, 159), (435, 1118), (377, 55)]


personCoordListCOCO = [(221, 543), (385, 88), (255, 371), (356, 220), (225, 121), (384, 57), (165, 534), (303, 207), (388, 354), (373, 68), (417, 135), (171, 7), (221, 12), (244, 333), (191, 63), (394, 588), (316, 584), (436, 314), (342, 53), (249, 32), (358, 380), (466, 190), (145, 231), (423, 203), (313, 19), (247, 80), (315, 621), (368, 69), (363, 161), (575, 394), (296, 475), (229, 571), (273, 68), (445, 439), (285, 145), (310, 77), (450, 394), (413, 173), (330, 241), (325, 84), (280, 279), (308, 129), (353, 488), (479, 281), (288, 64), (376, 148), (367, 157), (402, 51), (103, 572), (205, 482), (182, 558), (255, 412), (405, 252), (179, 369), (391, 224), (435, 321), (346, 453), (407, 605), (364, 590), (254, 355), (592, 330), (323, 466), (192, 83), (188, 554), (256, 106), (537, 442), (160, 24), (354, 442), (217, 158), (397, 6), (406, 54), (410, 340), (414, 548), (214, 51), (354, 177), (229, 257), (147, 72), (473, 441), (242, 303), (282, 532), (319, 280), (439, 12), (241, 101), (531, 80), (206, 572), (282, 421), (352, 262), (345, 432), (244, 299), (329, 107), (552, 92), (235, 309), (268, 74), (410, 32), (249, 468), (86, 624), (285, 239), (234, 319), (419, 218), (355, 588), (354, 6), (196, 79), (291, 97), (114, 225), (332, 120), (231, 560), (432, 292), (311, 80), (297, 358), (485, 266), (239, 385), (351, 326), (399, 22)]

carCoordListCOCO = [(358, 127), (376, 586), (257, 619), (331, 580), (295, 603), (359, 410), (253, 54), (163, 422), (242, 89), (418, 293), (289, 47), (459, 257), (371, 621), (169, 456), (327, 621), (406, 28), (464, 239), (180, 614), (411, 573), (332, 327), (385, 68), (139, 535), (357, 490), (415, 61), (449, 529), (308, 610), (371, 488), (574, 145), (453, 40), (345, 257), (233, 310), (207, 51), (418, 169), (247, 60), (326, 576), (329, 400), (145, 205), (329, 96), (381, 20), (362, 14), (302, 388), (288, 460), (511, 164), (231, 347), (392, 608), (351, 582), (272, 166), (305, 502), (237, 52), (251, 442), (408, 84), (332, 578), (437, 511), (419, 15), (557, 215), (253, 579), (243, 248), (592, 113), (600, 401), (264, 377), (236, 41), (205, 265), (192, 91), (428, 328), (284, 599), (304, 633), (303, 362), (439, 592), (197, 234), (380, 216), (400, 572), (249, 24), (216, 454), (194, 528), (359, 450), (332, 192), (258, 399), (428, 385), (427, 31), (280, 316), (196, 386), (97, 528), (213, 70), (255, 114), (315, 184), (526, 221), (344, 519), (439, 602), (443, 210), (461, 418), (234, 146), (159, 59), (369, 100), (325, 338), (144, 83), (399, 26), (226, 141), (297, 28), (292, 19), (389, 587), (298, 252), (129, 369), (360, 423), (319, 194), (307, 465), (259, 47), (445, 477), (299, 134), (281, 69), (207, 348), (225, 358), (94, 390), (198, 356), (496, 403), (180, 12), (368, 438), (174, 384), (429, 96), (317, 609), (376, 628), (175, 79), (438, 98), (341, 397)]

motorcycleCoordListCOCO = [(384, 347), (386, 44), (222, 213), (300, 217), (231, 433), (239, 281), (256, 302), (325, 471), (273, 400), (287, 372), (314, 346)]

truckCoordListCOCO = [(58, 326), (504, 371), (272, 351), (456, 496), (306, 621), (281, 474), (336, 330), (437, 85), (221, 351), (300, 230), (235, 457), (328, 302), (222, 71), (580, 278), (213, 578), (301, 132), (235, 357), (194, 306), (254, 19), (249, 195), (240, 344), (291, 464), (220, 368), (112, 522), (239, 304), (316, 328), (385, 432), (156, 28), (256, 355), (308, 320), (223, 369), (395, 200), (228, 125), (316, 15), (443, 219), (287, 156), (215, 137), (218, 343)]
'''



In [ ]:
#Iter 3

'''
personCoordList = [(468, 125), (440, 1694), (434, 759), (445, 1605), (457, 554), (475, 2022), (420, 1213), (494, 59), (427, 162), (449, 624), (448, 1682), (414, 688), (456, 1415), (431, 669), (430, 1100), (506, 1626), (483, 883), (430, 1441), (481, 662), (419, 735), (474, 1786), (402, 1692), (447, 1874), (422, 489), (472, 534), (444, 346), (472, 1512), (465, 1283), (466, 739), (466, 1824), (374, 1207), (461, 842), (446, 1047), (440, 292), (515, 31), (456, 490), (469, 407), (473, 1176), (438, 1527), (500, 824), (466, 1389), (464, 1075), (454, 1753), (516, 1468), (452, 553), (443, 1484), (474, 285), (443, 1371), (482, 536), (445, 974), (448, 1727), (539, 487), (451, 1658), (439, 1033), (485, 924), (483, 1126), (441, 1511), (332, 916), (442, 1130), (401, 1810), (473, 328), (447, 1792), (400, 2022), (484, 107), (433, 228), (450, 1955), (406, 163), (480, 1841), (461, 1142), (544, 1605), (458, 766), (461, 905), (434, 1278), (448, 1381), (421, 1913), (455, 1686), (442, 987), (435, 511), (445, 267), (479, 1653), (454, 558), (447, 924), (439, 1315), (416, 1172), (425, 1396), (425, 1545), (443, 1994), (450, 558), (429, 1561), (421, 76), (405, 1141), (577, 988), (440, 1473), (434, 365), (475, 218), (429, 804), (461, 300), (448, 119), (436, 1492), (247, 615), (389, 1415), (466, 2017), (453, 372), (453, 1471), (418, 1176), (477, 17), (436, 1126), (484, 1144), (464, 603), (444, 1237), (478, 1765), (453, 1664), (459, 895), (446, 612), (434, 408), (466, 2008), (453, 1105), (429, 1227), (459, 783), (455, 815), (485, 484), (393, 1655), (368, 1775), (454, 1655), (480, 1347), (450, 1126), (447, 705), (463, 33), (425, 1132), (455, 1143), (479, 331), (461, 1082), (475, 666), (451, 1206), (437, 220), (463, 23), (457, 14), (452, 397), (433, 1296), (368, 285), (442, 1322), (513, 388), (436, 1250), (469, 973), (465, 135), (480, 204), (431, 1620), (489, 166), (450, 1151), (437, 1531), (473, 726), (430, 1456), (425, 1309), (432, 1975), (482, 324), (476, 1481), (488, 430), (425, 1340), (443, 1278), (500, 262), (436, 1546), (441, 1926), (463, 120), (439, 1496), (458, 1737), (451, 313)]

carCoordList = [(409, 1651), (567, 361), (635, 1207), (460, 1298), (485, 1098), (468, 1163), (660, 1811), (439, 425), (489, 1013), (531, 554), (461, 1412), (687, 1969), (457, 894), (433, 851), (581, 1863), (485, 767), (450, 1175), (472, 925), (513, 754), (536, 1454), (457, 980), (573, 1766), (446, 52), (488, 412), (460, 834), (487, 22), (649, 1023), (471, 580), (485, 1150), (609, 1868), (487, 795), (578, 1269), (565, 318), (515, 684), (369, 575), (471, 1762), (447, 1209), (431, 561), (580, 1852), (452, 1259), (441, 679), (501, 578), (496, 914), (551, 94), (620, 1893), (736, 1982), (454, 739), (606, 1840), (461, 1911), (431, 959), (524, 400), (719, 142), (435, 1934), (473, 393), (483, 1296), (463, 437), (427, 64), (395, 1871), (469, 1460), (614, 1291), (575, 113), (471, 1196), (445, 579), (496, 1028), (460, 1384), (567, 159), (471, 1043), (422, 1123), (485, 699), (541, 176), (633, 126), (456, 904), (459, 1058), (456, 1289), (445, 1288), (448, 871), (462, 1094), (463, 889), (497, 60), (514, 254), (490, 1021), (595, 1945), (628, 1809), (513, 198), (656, 1800), (578, 1797), (462, 1065), (473, 717), (491, 693), (558, 1701), (511, 1052), (459, 679), (547, 463), (446, 1424), (490, 1056), (521, 258), (498, 793), (567, 167), (475, 850), (561, 280), (537, 471), (475, 1835), (468, 1633), (455, 987), (479, 1488), (473, 1396), (469, 1054), (463, 1018), (492, 21), (503, 486), (482, 72), (462, 756), (449, 363), (481, 63), (461, 431), (490, 211), (435, 391), (459, 1178), (468, 1301), (483, 540), (516, 276), (461, 850), (545, 326), (478, 1131), (481, 1435), (463, 754), (470, 797), (486, 1445), (453, 1774), (535, 446), (622, 270), (521, 1679), (455, 786), (455, 1382), (466, 1159), (519, 1179), (539, 508), (496, 1642), (451, 596), (490, 953), (474, 89), (455, 1255), (470, 1050), (486, 133), (468, 1004), (445, 669), (476, 1053), (479, 188), (442, 1756), (485, 1053), (409, 1227), (477, 1024), (458, 716), (497, 1363), (477, 276), (464, 544), (464, 1455), (502, 1812), (482, 55), (588, 1895), (482, 1597), (480, 525), (498, 1520), (522, 1489), (491, 596), (437, 889), (645, 29), (461, 210), (480, 1511), (462, 915), (611, 88), (487, 1961), (460, 907), (560, 1840), (481, 1470), (584, 30), (503, 604), (445, 1095), (480, 664), (525, 1515), (489, 1302), (692, 241), (684, 230), (517, 507), (583, 148), (474, 746), (463, 1312), (564, 281), (455, 929), (584, 1835), (506, 659), (628, 225), (675, 243), (592, 1938), (463, 841), (690, 1961), (448, 1221), (498, 650), (542, 122), (445, 1086), (574, 37), (532, 202), (440, 1205), (458, 967), (575, 1841), (507, 1350), (507, 1592), (457, 1113), (543, 126), (594, 1873), (464, 473), (466, 1013), (445, 864), (456, 890), (535, 1795), (532, 1787), (474, 910), (461, 914), (528, 1560), (587, 1776), (545, 459), (447, 801), (463, 430), (572, 1300), (474, 1081), (526, 506), (464, 1090), (470, 884), (427, 1391), (551, 2026), (456, 1583), (415, 448), (456, 1166), (458, 575), (424, 367), (450, 1058), (542, 289), (444, 1081), (715, 1971), (543, 1973), (693, 69), (548, 75), (466, 1496), (526, 1439), (507, 211), (454, 861), (552, 1885), (487, 1041), (460, 790), (516, 943), (493, 316), (445, 323), (495, 367), (470, 547), (461, 874), (442, 1969), (589, 1786), (484, 792), (488, 1063), (472, 812), (452, 1330), (442, 1146), (462, 927), (461, 773), (478, 1070), (486, 1136), (474, 1104), (461, 724), (482, 1088), (508, 653), (464, 48), (462, 778), (473, 977), (442, 1244), (475, 744), (448, 331)]

motorcycleCoordList = [(520, 1415), (509, 1050), (527, 1589), (545, 1435), (471, 233), (496, 1219), (556, 217), (611, 1759), (470, 1356), (484, 1301), (503, 1428), (510, 1371), (520, 524), (503, 1660), (485, 498), (510, 239), (495, 985), (479, 1797), (443, 1984)]

truckCoordList = [(412, 1374), (490, 145), (393, 1397), (431, 1360), (451, 688), (449, 1273), (439, 1538), (414, 438), (458, 671), (454, 1130), (485, 163), (435, 941), (432, 1251), (446, 306), (437, 855), (393, 1711), (409, 1209), (376, 1835), (420, 565), (438, 745), (430, 1598), (455, 778), (418, 443), (427, 951), (424, 482), (445, 1171), (418, 1128), (522, 1782), (439, 1023), (379, 1397), (428, 195), (415, 968), (383, 517), (417, 369), (449, 1864), (485, 1807), (629, 159), (435, 1118), (377, 55)]


personCoordListCOCO = [(221, 543), (415, 232), (255, 371), (356, 220), (218, 45), (311, 229), (173, 618), (186, 560), (388, 354), (373, 68), (417, 135), (171, 7), (221, 12), (244, 333), (232, 164), (444, 353), (316, 584), (436, 314), (335, 160), (249, 32), (358, 380), (466, 190), (154, 191), (423, 203), (313, 19), (247, 80), (315, 621), (368, 69), (367, 137), (575, 394), (159, 185), (229, 571), (273, 68), (445, 439), (285, 145), (310, 77), (450, 394), (413, 173), (330, 241), (329, 146), (280, 279), (308, 129), (353, 488), (479, 281), (288, 64), (376, 148), (317, 581), (402, 51), (346, 310), (205, 482), (160, 624), (255, 412), (405, 117), (179, 369), (391, 224), (435, 321), (344, 397), (407, 605), (364, 590), (254, 355), (592, 330), (323, 466), (223, 350), (188, 554), (269, 7), (517, 351), (160, 24), (354, 442), (217, 158), (397, 6), (406, 54), (410, 340), (414, 548), (214, 12), (354, 177), (229, 257), (130, 397), (473, 441), (242, 303), (282, 532), (314, 419), (439, 12), (241, 101), (531, 80), (206, 572), (292, 263), (352, 262), (357, 581), (330, 88), (330, 621), (570, 33), (194, 193), (268, 74), (410, 32), (276, 8), (86, 624), (290, 134), (234, 319), (419, 218), (355, 588), (354, 6), (196, 79), (291, 97), (114, 225), (332, 120), (272, 335), (432, 292), (311, 80), (315, 41), (485, 266), (239, 385), (351, 326), (399, 22)]

carCoordListCOCO = [(358, 127), (376, 586), (257, 619), (331, 580), (295, 603), (359, 410), (253, 54), (163, 422), (242, 89), (418, 293), (289, 47), (459, 257), (371, 621), (169, 456), (327, 621), (406, 28), (464, 239), (160, 513), (411, 573), (332, 327), (385, 68), (139, 535), (357, 490), (415, 61), (449, 529), (308, 610), (371, 488), (574, 145), (453, 40), (345, 257), (233, 310), (325, 577), (418, 169), (247, 60), (326, 576), (329, 400), (145, 205), (329, 96), (381, 20), (362, 14), (302, 388), (288, 460), (511, 164), (231, 347), (380, 433), (351, 582), (272, 166), (305, 502), (237, 52), (251, 442), (408, 84), (332, 578), (437, 511), (419, 15), (557, 215), (253, 579), (313, 590), (592, 113), (600, 401), (264, 377), (236, 41), (205, 265), (192, 91), (428, 328), (274, 23), (304, 633), (303, 362), (439, 592), (197, 234), (380, 216), (331, 114), (249, 24), (216, 454), (194, 528), (359, 450), (332, 192), (258, 399), (428, 385), (427, 31), (280, 316), (196, 386), (97, 528), (213, 70), (255, 114), (315, 184), (526, 221), (344, 519), (439, 602), (443, 210), (461, 418), (234, 146), (159, 59), (369, 100), (325, 338), (144, 83), (399, 26), (312, 18), (297, 28), (292, 19), (389, 587), (298, 252), (129, 369), (360, 423), (319, 194), (307, 465), (259, 47), (445, 477), (299, 134), (281, 69), (207, 348), (225, 358), (94, 390), (198, 356), (496, 403), (180, 12), (368, 438), (368, 475), (429, 96), (317, 609), (376, 628), (175, 79), (438, 98), (341, 397)]

motorcycleCoordListCOCO = [(384, 347), (386, 44), (222, 213), (300, 217), (231, 433), (285, 414), (256, 302), (325, 471), (244, 603), (287, 372), (314, 346)]

truckCoordListCOCO = [(58, 326), (504, 371), (272, 351), (456, 496), (306, 621), (281, 474), (336, 330), (437, 85), (221, 351), (300, 230), (235, 457), (328, 302), (222, 71), (580, 278), (213, 578), (301, 132), (235, 357), (210, 612), (254, 19), (249, 195), (240, 344), (291, 464), (220, 368), (112, 522), (239, 304), (316, 328), (385, 432), (156, 28), (256, 355), (308, 320), (223, 369), (395, 200), (228, 125), (316, 15), (443, 219), (287, 156), (215, 137), (218, 343)]
'''

In [ ]:
#Iter 4

'''
personCoordList = [(468, 125), (440, 1694), (439, 971), (445, 1605), (457, 554), (435, 41), (420, 1213), (450, 205), (427, 162), (456, 1207), (488, 251), (414, 688), (456, 1415), (431, 669), (430, 1100), (504, 1523), (483, 883), (453, 1324), (481, 662), (419, 735), (432, 632), (402, 1692), (447, 1874), (422, 429), (472, 534), (437, 1930), (455, 1970), (465, 1283), (535, 1987), (466, 1824), (377, 1159), (448, 1370), (446, 1047), (467, 1339), (510, 199), (474, 531), (449, 1217), (473, 1176), (438, 1527), (500, 824), (441, 1145), (464, 1075), (468, 1343), (489, 1668), (452, 553), (449, 1445), (438, 1086), (443, 1371), (482, 536), (429, 763), (448, 1727), (539, 487), (451, 1658), (450, 877), (485, 924), (475, 1164), (465, 183), (332, 916), (442, 1130), (401, 1810), (391, 448), (433, 1631), (400, 2022), (484, 107), (433, 228), (483, 1755), (463, 1470), (470, 1994), (455, 729), (478, 1765), (465, 704), (461, 905), (434, 1278), (448, 1381), (421, 1913), (455, 1686), (442, 987), (435, 511), (445, 267), (479, 1653), (454, 558), (447, 924), (439, 1315), (417, 1059), (425, 1396), (421, 1471), (443, 1994), (450, 558), (423, 1520), (421, 76), (405, 1141), (577, 988), (465, 1423), (434, 365), (475, 218), (431, 1282), (461, 300), (446, 170), (436, 1492), (247, 615), (389, 1415), (438, 1611), (453, 372), (453, 1471), (450, 264), (477, 17), (436, 1126), (484, 1144), (464, 603), (444, 1237), (478, 1765), (453, 1664), (473, 858), (446, 612), (457, 1475), (489, 1660), (483, 743), (429, 1227), (459, 783), (441, 1149), (485, 484), (393, 1655), (368, 1775), (454, 1655), (480, 1347), (447, 1099), (447, 705), (452, 559), (421, 1270), (447, 1236), (479, 331), (460, 977), (475, 666), (460, 1457), (437, 220), (435, 1701), (457, 1319), (452, 397), (436, 1263), (368, 285), (442, 1322), (513, 388), (436, 1250), (469, 973), (460, 40), (500, 2010), (426, 1246), (489, 166), (450, 1151), (437, 1531), (468, 1198), (430, 1456), (425, 1919), (438, 1844), (482, 324), (468, 1560), (488, 430), (425, 1574), (443, 1278), (501, 314), (436, 1546), (441, 1926), (463, 120), (439, 1496), (458, 1737), (426, 782)]

carCoordList = [(409, 1651), (567, 361), (635, 1207), (648, 184), (485, 1098), (468, 1163), (467, 1253), (439, 425), (489, 1013), (456, 1435), (461, 1412), (687, 1969), (475, 506), (434, 1802), (581, 1863), (485, 767), (450, 1175), (472, 925), (466, 1549), (534, 869), (459, 1231), (573, 1766), (446, 52), (622, 1807), (460, 834), (480, 768), (649, 1023), (471, 580), (485, 1150), (609, 1868), (487, 795), (582, 1483), (492, 894), (469, 1092), (369, 575), (471, 1762), (447, 1209), (431, 561), (490, 1431), (447, 597), (441, 679), (501, 578), (496, 914), (551, 94), (477, 1015), (485, 1106), (454, 739), (479, 939), (461, 1911), (431, 959), (524, 400), (719, 142), (446, 194), (477, 699), (533, 198), (605, 848), (465, 836), (395, 1871), (469, 1460), (634, 16), (486, 1159), (466, 1496), (469, 20), (496, 1028), (460, 1384), (451, 856), (536, 1790), (422, 1123), (459, 1055), (541, 176), (507, 1561), (506, 501), (462, 660), (453, 1099), (459, 1179), (448, 871), (462, 1094), (635, 1824), (506, 371), (471, 956), (441, 703), (451, 1340), (628, 1809), (478, 797), (669, 218), (578, 1797), (488, 1393), (473, 717), (470, 131), (558, 1701), (484, 1841), (470, 1352), (451, 960), (446, 1424), (466, 639), (451, 1094), (498, 793), (567, 167), (618, 1757), (427, 983), (453, 1011), (475, 1835), (468, 1633), (455, 987), (525, 676), (559, 1801), (491, 668), (475, 395), (548, 1608), (460, 1383), (516, 347), (462, 756), (449, 363), (481, 63), (461, 431), (471, 760), (454, 1500), (510, 1435), (509, 127), (421, 1903), (466, 862), (474, 1067), (475, 794), (478, 1131), (472, 1043), (583, 1889), (470, 797), (486, 1445), (453, 1774), (535, 446), (571, 1691), (515, 933), (473, 1190), (455, 1382), (466, 1159), (643, 94), (539, 508), (496, 1642), (459, 413), (482, 1622), (467, 468), (464, 517), (470, 1050), (457, 1061), (468, 1004), (445, 669), (476, 1053), (531, 1517), (442, 1756), (503, 1413), (409, 1227), (519, 1388), (458, 716), (497, 1363), (466, 44), (464, 544), (464, 1455), (502, 1812), (482, 55), (588, 1895), (463, 883), (530, 1312), (498, 1520), (460, 860), (491, 596), (450, 126), (645, 29), (454, 454), (480, 1511), (462, 915), (611, 88), (514, 844), (473, 696), (560, 1840), (481, 1470), (584, 30), (503, 604), (490, 1869), (522, 1556), (464, 747), (489, 1302), (577, 1597), (684, 230), (517, 507), (583, 148), (474, 746), (540, 1816), (564, 281), (526, 1830), (584, 1835), (591, 1813), (639, 1843), (491, 1502), (598, 289), (511, 1559), (594, 63), (448, 1221), (498, 650), (542, 122), (445, 1086), (472, 710), (488, 602), (452, 969), (488, 545), (575, 1841), (461, 1062), (507, 1592), (543, 152), (543, 126), (594, 1873), (589, 1855), (460, 1275), (487, 491), (456, 890), (484, 379), (427, 1047), (474, 910), (461, 914), (458, 720), (515, 337), (545, 459), (496, 40), (447, 1176), (572, 1300), (480, 222), (467, 1090), (464, 1090), (470, 884), (427, 1391), (551, 2026), (488, 1945), (415, 448), (523, 288), (415, 1938), (512, 1891), (556, 389), (542, 289), (483, 1772), (336, 628), (451, 710), (444, 1549), (548, 75), (434, 1092), (526, 1439), (507, 211), (454, 861), (552, 1885), (511, 232), (460, 790), (516, 943), (577, 89), (453, 884), (495, 367), (470, 547), (461, 874), (442, 1969), (589, 1786), (484, 792), (527, 87), (472, 812), (452, 1330), (611, 183), (462, 927), (444, 1471), (478, 1070), (486, 1136), (474, 1104), (453, 1087), (482, 1088), (431, 1782), (464, 48), (569, 125), (455, 645), (442, 1244), (465, 1325), (429, 754)]

motorcycleCoordList = [(520, 1415), (794, 77), (527, 1589), (545, 1435), (471, 233), (496, 1219), (556, 217), (611, 1759), (470, 1356), (484, 1301), (503, 1428), (510, 1371), (520, 524), (503, 1660), (485, 498), (510, 239), (495, 985), (479, 1797), (443, 1984)]

truckCoordList = [(412, 1374), (490, 145), (393, 1397), (431, 1360), (451, 688), (449, 1273), (439, 1538), (414, 438), (458, 671), (454, 1130), (485, 163), (435, 941), (432, 1251), (446, 306), (437, 855), (393, 1711), (409, 1209), (376, 1835), (420, 565), (438, 745), (430, 1598), (455, 778), (418, 443), (427, 951), (424, 482), (439, 858), (418, 1128), (522, 1782), (439, 1023), (379, 1397), (428, 195), (415, 968), (383, 517), (417, 369), (449, 1864), (485, 1807), (629, 159), (435, 1118), (377, 55)]


personCoordListCOCO = [(119, 31), (386, 13), (255, 371), (356, 220), (218, 45), (311, 229), (173, 618), (303, 207), (388, 354), (373, 68), (411, 9), (171, 7), (197, 156), (244, 333), (285, 545), (444, 353), (316, 584), (436, 314), (340, 102), (249, 32), (358, 380), (466, 190), (154, 191), (423, 203), (313, 19), (247, 80), (315, 621), (368, 69), (367, 137), (575, 394), (159, 185), (229, 571), (273, 68), (445, 439), (201, 377), (310, 77), (450, 394), (413, 173), (296, 496), (329, 146), (280, 279), (309, 27), (353, 488), (479, 281), (288, 64), (376, 148), (367, 157), (402, 51), (346, 310), (205, 482), (160, 624), (255, 412), (405, 117), (179, 369), (391, 224), (435, 321), (346, 453), (407, 605), (364, 590), (254, 355), (592, 330), (323, 466), (223, 350), (188, 554), (257, 33), (537, 442), (160, 24), (343, 594), (217, 158), (397, 6), (406, 54), (410, 340), (414, 548), (207, 108), (352, 72), (229, 257), (147, 72), (473, 441), (209, 556), (282, 532), (314, 419), (439, 12), (241, 101), (531, 80), (206, 572), (292, 263), (352, 262), (297, 393), (244, 299), (329, 107), (529, 147), (235, 309), (268, 74), (410, 32), (254, 431), (86, 624), (285, 239), (234, 319), (402, 385), (355, 588), (354, 6), (196, 79), (291, 97), (114, 225), (332, 120), (273, 109), (435, 181), (311, 80), (315, 41), (485, 266), (296, 227), (282, 31), (399, 22)]

carCoordListCOCO = [(358, 127), (376, 586), (257, 619), (347, 471), (295, 603), (359, 410), (253, 54), (163, 422), (242, 89), (418, 293), (288, 619), (459, 257), (371, 621), (169, 456), (327, 621), (347, 427), (454, 110), (160, 513), (411, 573), (332, 327), (385, 68), (139, 535), (357, 490), (415, 61), (449, 529), (308, 610), (371, 488), (574, 145), (453, 40), (345, 257), (233, 310), (325, 577), (418, 169), (247, 60), (326, 576), (329, 400), (145, 205), (329, 96), (381, 20), (362, 14), (302, 388), (288, 460), (511, 164), (231, 347), (380, 433), (351, 582), (272, 166), (305, 502), (237, 52), (251, 442), (408, 84), (332, 578), (437, 511), (419, 15), (557, 215), (253, 579), (243, 248), (592, 113), (600, 401), (264, 377), (236, 41), (205, 265), (192, 91), (428, 328), (274, 23), (304, 633), (303, 362), (427, 360), (197, 234), (380, 216), (400, 572), (249, 24), (216, 454), (194, 528), (359, 450), (332, 192), (258, 399), (428, 385), (427, 31), (280, 316), (196, 386), (97, 528), (213, 70), (443, 279), (304, 543), (526, 221), (344, 519), (439, 602), (443, 210), (461, 418), (234, 146), (159, 59), (369, 100), (346, 23), (144, 83), (399, 26), (226, 141), (297, 28), (292, 19), (375, 322), (298, 252), (129, 369), (360, 423), (319, 194), (307, 465), (259, 47), (445, 477), (299, 134), (281, 69), (207, 348), (225, 358), (94, 390), (198, 356), (496, 403), (180, 12), (368, 438), (399, 149), (450, 311), (317, 609), (376, 628), (175, 79), (438, 98), (341, 397)]

motorcycleCoordListCOCO = [(384, 347), (386, 44), (222, 213), (300, 217), (231, 433), (239, 281), (256, 302), (325, 471), (244, 603), (287, 372), (314, 346)]

truckCoordListCOCO = [(58, 326), (504, 371), (272, 351), (456, 496), (306, 621), (281, 474), (336, 330), (437, 85), (221, 351), (300, 230), (235, 457), (328, 302), (222, 71), (580, 278), (213, 578), (301, 132), (235, 357), (210, 612), (254, 19), (249, 195), (240, 344), (291, 464), (220, 368), (112, 522), (239, 304), (316, 328), (385, 432), (156, 28), (256, 355), (308, 320), (223, 369), (395, 200), (228, 125), (316, 15), (443, 219), (287, 156), (215, 137), (218, 343)]
'''

In [ ]:
#Iter 5


personCoordList = [(468, 125), (440, 1694), (434, 759), (445, 1605), (457, 554), (435, 41), (420, 1213), (452, 863), (427, 162), (436, 1675), (481, 335), (422, 729), (408, 960), (445, 986), (430, 1100), (504, 1523), (483, 883), (495, 423), (481, 662), (419, 735), (429, 588), (402, 1692), (385, 2011), (407, 118), (472, 534), (394, 1486), (463, 1613), (463, 1033), (466, 739), (466, 1824), (364, 1554), (448, 1370), (446, 1047), (440, 292), (515, 31), (474, 531), (449, 1217), (473, 1176), (438, 1527), (500, 824), (450, 1313), (464, 1075), (468, 1343), (516, 1468), (452, 553), (438, 1582), (482, 440), (445, 1741), (482, 536), (433, 1016), (448, 1727), (539, 487), (451, 1658), (439, 1033), (485, 924), (483, 1126), (441, 1511), (332, 916), (496, 1430), (401, 1810), (473, 328), (447, 1792), (400, 2022), (484, 107), (486, 92), (458, 11), (484, 1426), (480, 1841), (455, 729), (470, 1202), (465, 704), (461, 905), (434, 1278), (448, 1381), (421, 1913), (455, 1686), (442, 987), (435, 511), (445, 267), (453, 1374), (456, 481), (447, 924), (439, 1315), (416, 1172), (425, 1396), (421, 1471), (443, 1994), (450, 558), (423, 1520), (432, 439), (405, 1141), (577, 988), (440, 1473), (439, 666), (475, 218), (429, 804), (463, 1244), (446, 170), (436, 1492), (247, 615), (389, 1415), (466, 2017), (453, 372), (453, 1471), (450, 264), (477, 17), (436, 1126), (484, 1144), (464, 603), (444, 1237), (478, 1765), (453, 1664), (459, 895), (446, 612), (431, 1708), (466, 2008), (453, 1105), (429, 1227), (459, 783), (435, 1348), (485, 484), (393, 1655), (368, 1775), (454, 1655), (480, 1347), (447, 1099), (447, 705), (439, 638), (421, 1270), (455, 1143), (474, 216), (461, 197), (457, 1454), (452, 1610), (460, 2020), (437, 1425), (457, 1319), (452, 397), (433, 1296), (368, 285), (442, 1322), (513, 388), (436, 1250), (469, 973), (465, 135), (480, 204), (431, 1620), (489, 166), (450, 1151), (437, 1531), (466, 1264), (430, 1456), (425, 1919), (448, 1267), (482, 324), (476, 1481), (488, 430), (429, 1381), (443, 1278), (501, 314), (436, 1546), (441, 1926), (466, 51), (439, 1496), (458, 1737), (426, 782)]

carCoordList = [(445, 1163), (567, 361), (635, 1207), (498, 948), (485, 1098), (824, 2004), (467, 1253), (439, 425), (489, 1013), (531, 554), (461, 1412), (687, 1969), (457, 894), (433, 851), (581, 1863), (485, 767), (450, 1175), (472, 925), (466, 1549), (534, 869), (457, 980), (573, 1766), (446, 52), (622, 1807), (460, 834), (480, 768), (649, 1023), (471, 580), (485, 1150), (609, 1868), (487, 795), (582, 1483), (565, 318), (469, 1092), (369, 575), (471, 1762), (447, 1209), (431, 561), (490, 1431), (447, 597), (441, 679), (503, 1524), (496, 914), (531, 873), (477, 1015), (469, 721), (466, 1017), (479, 939), (461, 1911), (431, 959), (524, 400), (719, 142), (446, 194), (473, 393), (533, 198), (605, 848), (465, 836), (395, 1871), (469, 1460), (634, 16), (486, 1159), (515, 1835), (469, 20), (496, 1028), (467, 1095), (475, 714), (555, 47), (422, 1123), (459, 1055), (541, 176), (507, 1561), (506, 501), (459, 1058), (456, 1289), (445, 1288), (448, 871), (462, 1094), (463, 889), (506, 371), (481, 1694), (428, 617), (451, 1340), (509, 337), (478, 797), (656, 1800), (492, 1290), (462, 1065), (487, 1068), (470, 131), (511, 992), (490, 1336), (470, 1352), (451, 960), (446, 1424), (466, 639), (411, 1982), (495, 1204), (510, 1500), (475, 850), (561, 280), (453, 1011), (449, 838), (468, 1633), (513, 1512), (479, 1488), (559, 1801), (470, 197), (463, 1018), (548, 1608), (460, 1383), (469, 765), (462, 756), (449, 363), (481, 63), (461, 431), (470, 1799), (552, 812), (500, 941), (509, 127), (465, 1051), (481, 1401), (461, 850), (545, 326), (478, 1131), (444, 859), (463, 754), (470, 797), (486, 1445), (453, 1774), (535, 446), (622, 270), (515, 933), (455, 786), (455, 1382), (648, 88), (643, 94), (539, 508), (496, 1642), (451, 596), (482, 1622), (467, 468), (457, 1066), (470, 1050), (486, 133), (468, 1004), (519, 103), (476, 1053), (471, 1041), (442, 1756), (485, 1053), (409, 1227), (477, 1024), (458, 716), (497, 1363), (466, 44), (464, 544), (584, 1924), (502, 1812), (482, 55), (588, 1895), (458, 1401), (480, 525), (498, 1520), (460, 860), (491, 596), (437, 889), (645, 29), (440, 594), (480, 1511), (603, 1905), (611, 88), (514, 844), (495, 443), (475, 604), (612, 434), (478, 517), (494, 1435), (490, 1869), (480, 664), (525, 1515), (489, 1302), (692, 241), (684, 230), (517, 507), (583, 148), (474, 746), (463, 1312), (564, 281), (533, 140), (584, 1835), (506, 659), (628, 225), (675, 243), (598, 289), (511, 1559), (690, 1961), (448, 1221), (498, 650), (542, 122), (477, 803), (574, 37), (532, 202), (459, 806), (449, 1396), (575, 1841), (461, 1062), (507, 1592), (543, 152), (543, 126), (537, 137), (464, 473), (466, 1013), (487, 491), (465, 1290), (535, 1795), (428, 834), (474, 910), (452, 1374), (458, 720), (515, 337), (545, 459), (461, 400), (463, 430), (572, 1300), (445, 1425), (526, 506), (464, 1090), (470, 884), (427, 1391), (551, 2026), (522, 65), (415, 448), (456, 1166), (415, 1938), (424, 367), (450, 1058), (542, 289), (483, 1772), (336, 628), (543, 1973), (693, 69), (548, 75), (563, 129), (525, 227), (507, 211), (454, 861), (460, 1102), (511, 232), (460, 790), (516, 943), (577, 89), (450, 725), (626, 1826), (470, 547), (461, 874), (459, 1060), (589, 1786), (455, 1298), (478, 1449), (472, 812), (452, 1330), (442, 1146), (479, 237), (480, 1919), (472, 787), (486, 1136), (474, 1104), (470, 436), (482, 1088), (508, 653), (464, 48), (462, 778), (461, 188), (442, 1244), (507, 204), (429, 754)]

motorcycleCoordList = [(520, 1415), (794, 77), (527, 1589), (545, 1435), (471, 233), (496, 1219), (556, 217), (611, 1759), (470, 1356), (484, 1301), (503, 1428), (510, 1371), (520, 524), (503, 1660), (485, 498), (510, 239), (495, 985), (479, 1797), (443, 1984)]

truckCoordList = [(412, 1374), (490, 145), (393, 1397), (431, 1360), (451, 688), (419, 594), (439, 1538), (414, 438), (458, 671), (454, 1130), (485, 163), (435, 941), (432, 1251), (446, 306), (437, 855), (393, 1711), (409, 1209), (376, 1835), (420, 565), (438, 745), (430, 1598), (455, 778), (418, 443), (427, 951), (424, 482), (439, 858), (418, 1128), (522, 1782), (439, 1023), (379, 1397), (430, 1610), (415, 968), (383, 517), (417, 369), (449, 1864), (485, 1807), (629, 159), (435, 1118), (377, 55)]


personCoordListCOCO = [(221, 543), (385, 88), (255, 371), (356, 220), (225, 121), (382, 417), (186, 106), (186, 560), (388, 354), (373, 68), (411, 9), (171, 7), (197, 156), (244, 333), (191, 63), (423, 485), (316, 584), (436, 314), (335, 160), (197, 99), (358, 380), (466, 190), (145, 231), (423, 203), (313, 19), (247, 80), (315, 621), (368, 69), (364, 302), (575, 394), (159, 185), (229, 571), (273, 68), (445, 439), (231, 289), (310, 77), (450, 394), (413, 173), (330, 241), (295, 431), (280, 279), (308, 129), (353, 488), (479, 281), (288, 64), (376, 148), (317, 581), (402, 51), (103, 572), (205, 482), (160, 624), (255, 412), (417, 599), (179, 369), (391, 224), (435, 321), (344, 397), (407, 605), (364, 590), (254, 355), (592, 330), (323, 466), (232, 161), (188, 554), (257, 33), (508, 292), (160, 24), (354, 442), (217, 158), (397, 6), (414, 594), (410, 340), (414, 548), (207, 108), (354, 177), (229, 257), (130, 397), (473, 441), (209, 556), (282, 532), (314, 419), (439, 12), (241, 101), (538, 26), (206, 572), (282, 421), (352, 262), (297, 393), (244, 299), (330, 621), (570, 33), (190, 39), (268, 74), (410, 32), (260, 513), (86, 624), (290, 134), (234, 319), (402, 385), (315, 474), (354, 6), (196, 79), (291, 97), (114, 225), (332, 120), (272, 335), (435, 181), (311, 80), (315, 41), (485, 266), (304, 25), (300, 104), (399, 22)]

carCoordListCOCO = [(358, 127), (376, 586), (257, 619), (331, 580), (295, 603), (359, 410), (253, 54), (163, 422), (242, 89), (418, 293), (289, 47), (459, 257), (371, 621), (169, 456), (327, 621), (347, 427), (454, 110), (160, 513), (411, 573), (332, 327), (385, 68), (139, 535), (357, 490), (415, 61), (449, 529), (308, 610), (371, 488), (574, 145), (453, 40), (345, 257), (233, 310), (207, 51), (418, 169), (247, 60), (326, 576), (329, 400), (328, 82), (329, 96), (395, 573), (362, 14), (302, 388), (288, 460), (511, 164), (231, 347), (380, 433), (351, 582), (272, 166), (305, 502), (237, 52), (251, 442), (408, 84), (332, 578), (437, 511), (419, 15), (557, 215), (253, 579), (313, 590), (592, 113), (600, 401), (264, 377), (236, 41), (205, 265), (192, 91), (428, 328), (274, 23), (304, 633), (303, 362), (427, 360), (197, 234), (380, 216), (325, 375), (249, 24), (216, 454), (221, 61), (359, 450), (332, 192), (258, 399), (428, 385), (427, 31), (280, 316), (196, 386), (97, 528), (213, 70), (255, 114), (304, 543), (526, 221), (344, 519), (439, 602), (443, 210), (461, 418), (234, 146), (159, 59), (369, 100), (325, 338), (144, 83), (399, 26), (226, 141), (297, 28), (292, 19), (389, 587), (298, 252), (129, 369), (360, 423), (319, 194), (307, 465), (259, 47), (445, 477), (299, 134), (281, 69), (207, 348), (225, 358), (94, 390), (198, 356), (496, 403), (180, 12), (368, 438), (381, 35), (429, 96), (317, 609), (376, 628), (175, 79), (438, 98), (341, 397)]

motorcycleCoordListCOCO = [(384, 347), (386, 44), (222, 213), (300, 217), (231, 433), (239, 281), (256, 302), (325, 471), (273, 400), (287, 372), (314, 346)]

truckCoordListCOCO = [(58, 326), (504, 371), (272, 351), (456, 496), (306, 621), (281, 474), (336, 330), (437, 85), (221, 351), (300, 230), (235, 457), (328, 302), (222, 71), (580, 278), (213, 578), (301, 132), (235, 357), (210, 612), (254, 19), (249, 195), (240, 344), (291, 464), (220, 368), (112, 522), (239, 304), (316, 328), (385, 432), (156, 28), (256, 355), (308, 320), (223, 369), (395, 200), (228, 125), (316, 15), (443, 219), (287, 156), (215, 137), (218, 343)]


In [ ]:
import time
import math
import os

classes = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'stop sign', 'bird', 'cat', 'dog']
classes = ['person', 'car', 'motorcycle', 'truck']

classType = 'car'

#imgBlack = Image.new('RGB', (2048, 1024), color = 'black')
    
#imgWhite = Image.new('RGB', (2048, 1024), color = 'white')

timeFirst = time.perf_counter()

for classIndex, elemClass in enumerate(classes):
    
    print(elemClass)
    classCoordList = []
    
    strVal = str(classIndex+1).zfill(3)
    
    timeNow = time.perf_counter()
    
    countCoord = -1
    
    if (classIndex==0):
        
        refList = personCoordListCOCO
    elif (classIndex==1):
        
        refList = carCoordListCOCO
    elif (classIndex==2):
        
        refList = motorcycleCoordListCOCO
    elif (classIndex==3):
        
        refList = truckCoordListCOCO

    for i in range(300):
    
        #curNum = str(i).zfill(6)

        '''
        strName = "bremen_" + curNum + "_000019_leftImg8bit"
        #strName = "COCO" + str(i) 

        openName = "./BremenRotateOut/" + strName + "_mask" + strVal + ".png"
        #openName = "./COCORotateOut/" + strName + "_mask" + strVal + ".png"
        '''
        
        #strName = "bremen_" + curNum + "_000019_leftImg8bit"
        strName = "COCO" + str(i) 

        #openName = "./Rotate/Iter5/Iter5BremenRotateOut/" + strName + "_mask" + strVal + ".png"
        openName = "./Rotate/Iter5/Iter5COCORotateOut/" + strName + "_mask" + strVal + ".png"

        if os.path.exists(openName):
            
            countCoord = countCoord + 1

            background1 = scale_rotate_image(Image.open(openName, 'r'), img_scale = 1, img_rotate = 0)

            coordPoint = refList[countCoord]
            
            row, col = coordPoint

            '''
            strInsert = "./BremenRotateImages/" + strName + "_mask" + strVal + ".png"
            #strInsert = "./COCORotateImages/" + strName + "_mask" + strVal + ".png"
            '''
            
            #strInsert = "./Rotate/Iter5/Iter5BremenRotateImages/" + strName + "_mask" + strVal + ".png"
            strInsert = "./Rotate/Iter5/Iter5COCORotateImages/" + strName + "_mask" + strVal + ".png"
            
            resultIm = Image.open(strInsert)

            turn = random.randint(10, 60)
            randInt = random.randint(1,2)
            
            if (randInt==2):
                
                turn = 360 - turn

            resultIm = resultIm.rotate(turn, center=(col, row))

            background1.paste(resultIm, (0, 0), mask=resultIm)
            
            #saveStrName = "./Rotate/Iter5/Iter5BremenRotateFinal/" + strName + "_mask" + strVal + ".png"
            
            saveStrName = "./Rotate/Iter5/Iter5COCORotateFinal/" + strName + "_mask" + strVal + ".png"

            background1.save(saveStrName)
    
    timeEnd = time.perf_counter()
    elapTime = timeEnd - timeNow
    print(elapTime)

print("Total Time")
timeEnd = time.perf_counter()
elapTime = timeEnd - timeFirst
print(elapTime)
print("Done")
    